In [1]:
#[Pre-intervention] Retrieve data from the ENDORSE mobile apps 
import pandas as pd
#---------Function retrieving data from xlsx---------------------------------------
def retrieve_data(ExcelName, SheetName):
    df = pd.read_excel(io=ExcelName,sheet_name=SheetName)
    #Exclude dropouts
    df = df.loc[(df["pid"] != '607e774a3e642a744df77a42') & (df["pid"] != '6075345b45d8ce544ea3888b')]
    df.reset_index(drop=True)
    return df

def scores(df,ColumnNames):
    if df["Score - v"].dtypes == 'object':
        temp1 = df["Score - v"].str.split(",", n = -1, expand = True)
        temp2 = temp1.join(df['pid'])
    else:
        temp2 = df[["Score - v","pid"]]
    df = pd.DataFrame(temp2.values,columns = ColumnNames)
    return df

def generate_descriptive_statistics(df,Names,type_of_variable,conditions):
    if isinstance(df, pd.DataFrame)==False:
            df = pd.to_numeric(df,downcast = "float")
    else:
        for col in df.columns:
            df[col] = pd.to_numeric(df[col], downcast = "float")
    if type_of_variable == 'continous':
        average = round(df.mean(),2)
        SD = round(df.std(),2)
        if len(Names)==1:
            number = str(average) + '±' + str(SD)
        else:
            number = ''
            k=1
            for i in range(len(Names)):
                if k==1:
                    number = Names[i]+': '+str(average[i]) + ' ± ' + str(SD[i])
                    k=k+1
                else:
                    number = number + ', ' + Names[i]+': '+str(average[i]) + '±' + str(SD[i])
    elif type_of_variable == 'categorical1':
        for l in range(len(conditions)):
            if l==0:
                number = Names[l] + ': ' + str(len(df[(df>=conditions[l])&(df<conditions[l+1])])) + '(' + str(round(100*len(df[(df>=conditions[l])&(df<conditions[l+1])])/len(df),2)) + '%)'
            elif l==(len(conditions)-1):
                number = number + ', ' + Names[l] + ': ' + str(len(df[(df>=conditions[l])])) + '(' + str(round(100*len(df[(df>=conditions[l])])/len(df),2)) + '%)'
            elif l<(len(conditions)-1):
                number = number + ', ' + Names[l] + ': ' + str(len(df[(df>=conditions[l])&(df<conditions[l+1])])) + '(' + str(round(100*len(df[(df>=conditions[l])&(df<conditions[l+1])])/len(df),2)) + '%)'
    elif type_of_variable == 'categorical2':
        for l in range(len(conditions)):
            if l==0:
                number = Names[l] +': '+str(len(df[df==conditions[l]])) + '(' + str(round(100*len(df[df==conditions[l]])/len(df),2))+ '%)'
            else:
                number = number + ', ' + Names[l] + ': ' + str(len(df[df==conditions[l]])) + '(' + str(round(100*len(df[df==conditions[l]])/len(df),2))+ '%)'
    return number
        
def nlp(df,column_name,words,replaces):
    df_new = pd.DataFrame
    for i in range(len(words)):
        temp = df[column_name].str.find(words[i])
        df[column_name][(temp>-1)] = replaces[i]
    return df

#-------------------------------------Create Dataframes for descriptive statistics-----------------------------------
C = 'Χαρακτηριστικό'
N = 'Μέση τιμή ± Τυπική απόκλιση/Πλήθος'
dsdemographical = pd.DataFrame()
dssoma = pd.DataFrame()
dsscores = pd.DataFrame()
dsbiochemical = pd.DataFrame()
dshistory = pd.DataFrame()



In [ ]:
#Demographical
#----------------------------------------------------Gender-------------------------------------------------------------------------------
medical_history1 = retrieve_data('questionnaireData.xlsx','medial_clinical')
medical_history2 = retrieve_data('questionnaireData_ctrlpre.xlsx','medial_clinical')
medical_history = pd.concat([medical_history1, medical_history2], axis=0)
medical_history.drop(['Unnamed: 0'],axis=1,inplace=True)
medical_history.reset_index(drop=True, inplace=True)
s = pd.notnull(medical_history["Όγκος όρχεων Αρ - v"])
Males_temp = medical_history[s]
Females_temp = medical_history[~s]
Males = Males_temp["pid"]
Females = Females_temp["pid"]
sex = pd.DataFrame({'Gender':[],'pid':[]})
for i in range(len(medical_history)):
    ids = medical_history["pid"][i]
    if s[i]==True:
        sex = sex.append({'Gender':'M','pid':ids}, ignore_index = True)
    else:
        sex = sex.append({'Gender':'F','pid':ids}, ignore_index = True)  
number = 'Αγόρια: '+ str(len(Males))+ ', ' + 'Κορίτσια: ' + str(len(Females))
characteristic = 'Φύλο'
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)
#---------------------------------------------------------------------------------------------------------------------

#------------------------------------------------------Age----------------------------------------------------------------------------------
from datetime import datetime,date
temp_age1 = retrieve_data('questionnaireData.xlsx','medical_prebirth')
temp_age2 = retrieve_data('questionnaireData_ctrlpre.xlsx','medical_prebirth')
temp_age = pd.concat([temp_age1, temp_age2], axis=0)
temp_age.drop(['Unnamed: 0'],axis=1,inplace=True)
temp_age.reset_index(drop=True, inplace=True)

temp_age = temp_age[["ΗΜΕΡΟΜΗΝΙΑ ΓΕΝΝΗΣΗΣ - t","ΗΜΕΡΟΜΗΝΙΑ ΚΛΙΝΙΚΗΣ ΕΞΕΤΑΣΗΣ - t","pid"]]
clinical_date = temp_age[["ΗΜΕΡΟΜΗΝΙΑ ΚΛΙΝΙΚΗΣ ΕΞΕΤΑΣΗΣ - t"]]
birth_date = temp_age[["ΗΜΕΡΟΜΗΝΙΑ ΓΕΝΝΗΣΗΣ - t"]]
rows=[]
for i in range(len(clinical_date)):
    born = datetime.strptime(birth_date.loc[i][0],"%d/%m/%Y").date()
    clinical = datetime.strptime(clinical_date.loc[i][0],"%d/%m/%Y").date()
    age = clinical.year - born.year - ((clinical.month,clinical.day) < (born.month,born.day))
    rows.append([age,temp_age["pid"][i]])
Age = pd.DataFrame(rows,columns = ["Age","pid"])
characteristic = 'Ηλικία παιδιού'
number = generate_descriptive_statistics(Age["Age"],['Test'],'continous',[])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)
#------------------------------------------------------------------------------------------------------------------

#-------------------------------------------------Koinwnikodimografika xarakthristika oikogeneias----------------------------------------------
medical_family1 = retrieve_data('questionnaireData.xlsx','medical_family')
medical_family2 = retrieve_data('questionnaireData_ctrlpre.xlsx','medical_family')
medical_family = pd.concat([medical_family1, medical_family2], axis=0)
medical_family.drop(['Unnamed: 0'],axis=1,inplace=True)
medical_family.reset_index(drop=True, inplace=True)

BMI_m_f = medical_family["BMI - v"].str.split(",", n = -1, expand = True)
BMI_m_f = pd.DataFrame(BMI_m_f.values,columns = ["BMI_mother","BMI_Father"])
medical_family_mother = medical_family[["pid","ΗΛΙΚΙΑ ΜΗΤΡΟΣ - v",
                                        "ΥΨΟΣ ΜΗΤΡΟΣ - v","ΒΑΡΟΣ ΜΗΤΡΟΣ - v",
                                        "ΕΚΠΑΙΔΕΥΣΗ ΜΗΤΡΟΣ - v",
                                        "ΕΠΑΓΓΕΛΜΑ ΜΗΤΡΟΣ - v",
                                        "ΝΟΣΗΜΑΤΑ ΜΗΤΡΟΣ - v"]]#0:Kanena,1:Diabetes,2:Yperlipidaimia,3: Στεφανιαία νόσος, 4:Hypertension,5:Paidiki Pahisarkia Gonewn,6:Pathisi Thiroeidous
medical_family_mother = medical_family_mother.join(BMI_m_f.iloc[:,[0]])
medical_family_father = medical_family[["pid","ΗΛΙΚΙΑ ΠΑΤΡΟΣ - v",
                                        "ΥΨΟΣ ΠΑΤΡΟΣ - v",
                                        "ΒΑΡΟΣ ΠΑΤΡΟΣ - v",
                                        "ΕΚΠΑΙΔΕΥΣΗ ΠΑΤΡΟΣ - v",
                                        "ΕΠΑΓΓΕΛΜΑ ΠΑΤΡΟΣ - v",
                                        "ΝΟΣΗΜΑΤΑ ΠΑΤΡΟΣ - v"]]#0:Kanena,1:Diabetes,2:Yperlipidaimia,4:Hypertension,5:Paidiki Pahisarkia Gonewn,6:Pathisi Thiroeidous
medical_family_father = medical_family_father.join(BMI_m_f.iloc[:,[1]])

def mother_father(df_age,df_BMI,df_edu,df_dis,parent,dsdemographical):
    characteristic = 'Ηλικία '+ parent
    number = generate_descriptive_statistics(df_age,['Test'],'continous',[])
    dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)
    characteristic = 'ΔΜΣ ' + parent  
    Names = ['Ελλιποβαρής', 'Φυσιολογικό Βάρος','Υπερβαρότητα','Παχυσαρκία']
    conditions = [0,18.5,25,30]
    number = generate_descriptive_statistics(df_BMI,Names,'categorical1',conditions)
    dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

    characteristic = 'Εκπαίδευση ' + parent
    column_name = df_edu.columns[1]
    words = ['Γυμν','Λυκ','ΑΕΙ','ΙΕΚ','ΤΕΙ']
    replaces = ['Απόφοιτος Γυμνασίου','Απόφοιτος Λυκείου','ΑΕΙ','ΙΕΚ','ΤΕΙ']
    education = nlp(df_edu,column_name,words,replaces)
    number = ''
    for i in replaces:
        temp = education[column_name].str.find(i)
        if i==replaces[0]:
            number = i + ': ' + str(len(temp[temp>-1])) + ' (' + str(round(100*len(temp[temp>-1])/len(temp),2)) + '%)'
        else:
            number = number + ', ' + i + ': ' + str(len(temp[temp>-1])) + '(' + str(round(100*len(temp[temp>-1])/len(temp),2))+'%)'
    dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

    characteristic = 'Νοσήματα ' + parent
    Names = ['Κανένα','Σακχαρώδης Διαβήτης','Υπερλιπιδαιμία','Στεφανιαία νόσος','Υπέρταση','Παιδική παχυσαρκία γονέων','Πάθηση Θυρεοειδούς']
    number = ''
    for i in range(len(Names)):
        temp = df_dis.str.find(str(i))
        if i == 0:
            number = Names[i] + ': ' +str(len(temp[temp>-1])) + ' (' +str(round(100*len(temp[temp>-1])/len(temp),2)) + '%)'
        else:
            number = number + ', ' + Names[i] + ': '+str(len(temp[temp>-1])) +' (' +str(round(100*len(temp[temp>-1])/len(temp),2)) + '%)'
    temp = df_dis.str.find('α' or 'ε' or 'η' or 'ο')  #Check it its wrong
    for i in range(len(temp[temp>-1])):
        l = temp[temp>-1].index[i]
        number = number + ', ' + df_dis[l]+': '+ "1"
    dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)
    return dsdemographical

df_age = medical_family_mother["ΗΛΙΚΙΑ ΜΗΤΡΟΣ - v"]
df_BMI = BMI_m_f["BMI_mother"]
df_edu = medical_family_mother[["pid","ΕΚΠΑΙΔΕΥΣΗ ΜΗΤΡΟΣ - v"]]
df_dis = medical_family_mother["ΝΟΣΗΜΑΤΑ ΜΗΤΡΟΣ - v"]
parent = 'μητρός'
dsdemographical = mother_father(df_age,df_BMI,df_edu,df_dis,parent,dsdemographical)
df_age = medical_family_father["ΗΛΙΚΙΑ ΠΑΤΡΟΣ - v"]
df_BMI = BMI_m_f["BMI_Father"]
df_edu = medical_family_father[["pid","ΕΚΠΑΙΔΕΥΣΗ ΠΑΤΡΟΣ - v"]]
df_dis = medical_family_father["ΝΟΣΗΜΑΤΑ ΠΑΤΡΟΣ - v"]
parent = 'πατρός'
dsdemographical = mother_father(df_age,df_BMI,df_edu,df_dis,parent,dsdemographical)

#0:Kanena,1:Diabetes,2:Yperlipidaimia,3: ?? [SOS] 4:Hypertension,5:Paidiki Pahisarkia Gonewn,6:Pathisi Thiroeidous
#[SOS] η στήλη ["ΝΟΣΗΜΑΤΑ ΠΑΤΡΟΣ - t"] λείπει
#--------------------------------------------------------------------------------------------------------------------------

#-------------------------------------Koinwnikodimografika charactiristika Miteras----------------------------------------
social_demo1 = retrieve_data('questionnaireData.xlsx',"socialdemo")
social_demo2 = retrieve_data('questionnaireData_ctrlpre.xlsx',"socialdemo")
social_demo = pd.concat([social_demo1, social_demo2], axis=0)
social_demo.drop(['Unnamed: 0'],axis=1,inplace=True)
social_demo.reset_index(drop=True, inplace=True)

social_demo = social_demo[["pid",
                           "Ύψος σε μέτρα - v",
                           "Ασκείστε; Πόσο συχνά; - v",
                           "Βάρος σε κιλά - v",
                           "Είστε ικανοποιημένος με το εισόδημά σας; - v",
                           "Είστε καπνιστής; - v",
                           "Ηλικία - v",
                           "Καταναλώνετε αλκοολούχα ποτά; - v",
                           "Με βάση το ύψος σας και το σωματικό σας βάρος, θεωρείτε πως είστε - v",
                           "Νοσείτε από κάποια χρόνια ασθένεια (πχ. Υπέρταση, Δυσλιπιδαιμία, Διαβήτη,); - v",
                           "Παρακαλούμε διευκρινίστε την ασθένεια - v",
                           "Ποια είναι η εθνικότητά σας*; - v",
                           "Ποια είναι η εργασία σας; (σε περίπτωση συνταξιοδότησης σημειώνετε την απασχόληση πριν από τη συνταξιοδότηση) - v",
                           "Ποια είναι η οικογενειακή σας κατάσταση; - v",
                           "Ποιο είναι το ακαθάριστο ετήσιο εισόδημά σας; (η ερώτηση αυτή είναι προαιρετική) - v",
                           "Ποιο είναι το μορφωτικό σας επίπεδο; - v",
                           "Πόσο καλή είναι η υγεία σας; - v",
                           "Το οικογενειακό σας εισόδημα καλύπτει τις ανάγκες σας; - v",
                           "Φύλο - v"]]

characteristic = 'Σωματική δραστηριότητα γονέα'
number = generate_descriptive_statistics(social_demo["Ασκείστε; Πόσο συχνά; - v"],['Καθόλου', '1-3 φορές το μήνα', '1 φορά την εβδομάδα', '2-4 φορές την εβδομάδα','5-6 φορές την εβδομάδα','1 φορά την ημέρα'],'categorical2',[0,1,2,3,4,5])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Ακαθάριστο εισόδημα γονέα'
number = generate_descriptive_statistics(social_demo["Ποιο είναι το ακαθάριστο ετήσιο εισόδημά σας; (η ερώτηση αυτή είναι προαιρετική) - v"],['<10000 €', '10.000-20.000 €','20.000-30.000 €','30.000-40.000 €','> 40.000 €'],'categorical2',[0,1,2,3,4])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Ικανοποίηση γονέα με το εισόδημα'
number = generate_descriptive_statistics(social_demo["Είστε ικανοποιημένος με το εισόδημά σας; - v"],['Καθόλου', 'Μέτρια', 'Πολύ'],'categorical2',[0,1,2])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Κάπνισμα γονέα'
number = generate_descriptive_statistics(social_demo["Είστε καπνιστής; - v"],['Ναι', 'Όχι'],'categorical2',[0,1])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Κατανάλωση αλκοολούχων ποτών γονέα'
number = generate_descriptive_statistics(social_demo["Καταναλώνετε αλκοολούχα ποτά; - v"],['Καθόλου', '1-3 φορές το μήνα','1 φορά την εβδομάδα'],'categorical2',[0,1,2])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Επίγνωση γονέα σχετικά με την κατηγορία που ανήκει σύμφωνα με το ΔΜΣ'
number = generate_descriptive_statistics(social_demo["Με βάση το ύψος σας και το σωματικό σας βάρος, θεωρείτε πως είστε - v"],['Φυσιολογικός/η', 'Υπέρβαρος/η','Παχύσαρκος/η'],'categorical2',[1,2,3])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Χρόνια ασθένεια γονέα'
number = generate_descriptive_statistics(social_demo["Νοσείτε από κάποια χρόνια ασθένεια (πχ. Υπέρταση, Δυσλιπιδαιμία, Διαβήτη,); - v"],['Ναι', 'Όχι'],'categorical2',[0,1])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Εθνικότητα γονέα'
number = generate_descriptive_statistics(social_demo["Ποια είναι η εθνικότητά σας*; - v"],['Ελληνική', 'Αλβανική','Πολωνική','Βουλγαρική','Ρουμανική','Άλλη'],'categorical2',[0,1,2,3,4,5])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Εργασία γονέα'
temp = social_demo["Ποια είναι η εργασία σας; (σε περίπτωση συνταξιοδότησης σημειώνετε την απασχόληση πριν από τη συνταξιοδότηση) - v"]
temp2 = temp.astype(str).str.isnumeric()
number = generate_descriptive_statistics(temp.loc[temp2==True],['Ανώτερο διοικητικό στέλεχος (δημόσιου και ιδιωτικού τομέα)', 'Δημόσιος  - Ιδιωτικός Υπάλληλος (επιστήμονα)','Δημόσιος  - Ιδιωτικός Υπάλληλος (υπάλληλος γραφείου – μη επιστήμονας)','Δημόσιος – Ιδιωτικός-Ελεύθερος Επαγγελματίας, Επαγγελματίας Υγείας (ιατρός, νοσηλευτής, διαιτολόγος, φυσιοθεραπευτής κα.)','Έμπορος','Ελεύθερος επαγγελματίας','Εξειδικευμένος τεχνίτης','Ανειδίκευτος εργάτης','Οικιακή παραγωγή (Οικιακά)','Άλλο'],'categorical2',[0,1,2,3,4,5,6,7,8,9])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Οικογενειακή κατάσταση γονέα'
number = generate_descriptive_statistics(social_demo["Ποια είναι η οικογενειακή σας κατάσταση; - v"],['Άγαμος/η', 'Έγγαμος/η','Διαζευγμένος/η','Χήρος/α'],'categorical2',[0,1,2,3])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Μορφωτικό επίπεδο γονέα'
number = generate_descriptive_statistics(social_demo["Ποιο είναι το μορφωτικό σας επίπεδο; - v"],['Δημοτικό', 'Γυμνάσιο','Γενικό Λύκειο / Τεχνικό Επαγγελματικό Λύκειο','ΙΕΚ','ΤΕΙ','ΑΕΙ','Κάτοχος Μεταπτυχιακού διπλώματος','Κάτοχος Διδακτορικού διπλώματος'],'categorical2',[0,1,2,3,4,5,6,7])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Κατάσταση υγείας γονέα'
number = generate_descriptive_statistics(social_demo["Πόσο καλή είναι η υγεία σας; - v"],['Κακή', 'Μέτρια','Καλή','Πολύ καλή','Εξαιρετική'],'categorical2',[0,1,2,3,4])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Κάλυψη αναγκών από το οικογενειακό εισόδημα'
number = generate_descriptive_statistics(social_demo["Το οικογενειακό σας εισόδημα καλύπτει τις ανάγκες σας; - v"],['Καθόλου', 'Λίγο','Μέτρια','Πολύ','Πάρα πολύ'],'categorical2',[0,1,2,3,4])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Φύλο γονέα'
number = generate_descriptive_statistics(social_demo["Φύλο - v"],['Άνδρας', 'Γυναίκα'],'categorical2',[0,1])
dsdemographical = dsdemographical.append({C:characteristic,N:number},ignore_index=True)

#--------------------------------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:8870: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
dsdemographical=dsdemographical.drop_duplicates()
dsdemographical

,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Φύλο,"Αγόρια: 23, Κορίτσια: 7"
1,Ηλικία παιδιού,10.0±2.01
2,Ηλικία μητρός,43.73±5.36
3,ΔΜΣ μητρός,"Ελλιποβαρής: 0(0.0%), Φυσιολογικό Βάρος: 11(24..."
4,Εκπαίδευση μητρός,"Απόφοιτος Γυμνασίου: 3 (6.67%), Απόφοιτος Λυκε..."
5,Νοσήματα μητρός,"Κανένα: 21 (46.67%), Σακχαρώδης Διαβήτης: 1 (2..."
6,Ηλικία πατρός,47.09±6.42
7,ΔΜΣ πατρός,"Ελλιποβαρής: 0(0.0%), Φυσιολογικό Βάρος: 6(13...."
8,Εκπαίδευση πατρός,"Απόφοιτος Γυμνασίου: 8 (17.78%), Απόφοιτος Λυκ..."
9,Νοσήματα πατρός,"Κανένα: 27 (60.0%), Σακχαρώδης Διαβήτης: 3 (6...."


In [ ]:
#Scores
#--------------------------------------------------Food Practices --------------------------------------------------------------------------------
foodpractices1 = retrieve_data('questionnaireData.xlsx','foodpractices')
foodpractices2 = retrieve_data('questionnaireData_ctrlpre.xlsx','foodpractices')
foodpractices = pd.concat([foodpractices1, foodpractices2], axis=0)
foodpractices.drop(['Unnamed: 0'],axis=1,inplace=True)
foodpractices.reset_index(drop=True, inplace=True)

ColumnNames = ["Modeling","Coercive Control","Emotional Regulation/Food as reward","Healthy Eating Guidance",
               "Restriction","Pressure to eat","pid"] #Mon: Simainei Modeling, Con: Simainei control, Reg: simainei emotional regulation/food as reward, Guid: simainei healthy eating guidance,Res: simainei restriction, Pres: simainei Pressure to eat
foodpractices = scores(foodpractices,ColumnNames)
characteristic = 'Γονεϊκές πρακτικές σίτισης'
number = generate_descriptive_statistics(foodpractices[["Modeling","Coercive Control","Emotional Regulation/Food as reward","Healthy Eating Guidance",
               "Restriction","Pressure to eat"]],
                ['Γονείς-πρότυπα','Καταναγκαστικός έλεγχος','Χρήση του φαγητού ως επιβράβευση','Ενθάρρυνση',
                 'Περιοριστική καθοδήγηση','Πίεση για φαγητό'],'continous',[])
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)
#----------------------------------------------------------------------------------------------------------------------------

#-------------------------------------------------Depression score-----------------------------------------------------------
#[0 – 4]: None-minimal None
#[5 – 9]: Mild Watchful waiting; repeat PHQ-9 at follow-up
#[10 – 14]: Moderate Treatment plan, considering counseling, follow-up and/or pharmacotherapy
#[15 – 19]: Moderately Severe, Active treatment with pharmacotherapy and/or psychotherapy
#[20 – 27]: Severe, Immediate initiation of pharmacotherapy and, if severe impairment or poor response to therapy, expedited referral to a mental health specialist for psychotherapy and/or collaborative management
psycho1 = retrieve_data('questionnaireData.xlsx','psycho')
psycho2 = retrieve_data('questionnaireData_ctrlpre.xlsx','psycho')
psycho = pd.concat([psycho1, psycho2], axis=0)
psycho.drop(['Unnamed: 0'],axis=1,inplace=True)
psycho.reset_index(drop=True, inplace=True)

ColumnNames = ["Depressive Score","pid"]
psycho = scores(psycho,ColumnNames)

characteristic = 'Σκορ κατάθλιψης'
number = generate_descriptive_statistics(psycho["Depressive Score"],
                ['Test'],'continous',[])
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Κατηγορία σκορ κατάθλιψης'
temp = psycho["Depressive Score"]
AC = str(round(100*len(psycho[(temp>=0) & (temp<=4)])/len(psycho),2))
BC = str(round(100*len(psycho[(temp>=5) & (temp<=9)])/len(psycho),2))
CC = str(round(100*len(psycho[(temp>=10) & (temp<=14)])/len(psycho),2))
DC = str(round(100*len(psycho[(temp>=15) & (temp<=19)])/len(psycho),2))
EC = str(round(100*len(psycho[(temp>=20)])/len(psycho),2))
number = 'Κανένα σύμπτωμα: ' + str(len(psycho[(temp>=0) & (temp<=4)])) + '(' + AC + ')%' + ', Ελάχιστα συμπτώματα: ' + str(len(psycho[(temp>=5) & (temp<=9)])) + '('+ BC + ')%' +', Ελάσσονα κατάθλιψη, δυσθυμία, ήπια μείζονα κατάθλιψη: ' + str(len(psycho[(temp>=10) & (temp<=14)])) + '('+CC + ')%' +', Μέτρια έως σοβαρή μείζονα κατάθλιψη: ' + str(len(psycho[(temp>=15) & (temp<=19)]))+ '('+DC + ')%' + ', Σοβαρή μείζονα κατάθλιψη: ' + str(len(psycho[(temp>=20)])) + '('+ EC + ')%'
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)
#--------------------------------------------------------------------------------------------------------------------------

#-----------------------------------------Eating attitudes----------------------------------------------------
#To EAT-26 αποτελείται από 26 ερωτήσεις με 6 πιθανές απαντήσεις. 
#Η βαθμολογία κάθε ερώτησης κυμαίνεται από 0 έως 3 (3=συχνά, 2=συνήθως, 1=πάντα, 0=μερικές φορές, σπάνια, ποτέ),  
#με  συνέπεια  η  τελική  βαθμολογία  του  ερωτηματολογίου να κυμαίνεται από 0–78. 
#Η αξιολόγηση της εγκυρότητας του ερωτηματολογίου στα ελληνικά έχει γίνει από τον Σίμο κ.ά. (1996). 
#Το ΕΑΤ-26 αποτελείται από τρεις υποκλίμακες: Δίαιτα, Βουλιμία και Έλεγχος του φαγητού. 
#Στην ελληνική έκδοση του ερωτηματολογίου βαθμολογία ίση ή μεγαλύτερη από 20 θεωρείται ότι είναι δηλωτική υψηλού κινδύνου 
#για την εμφάνιση διαταραχής πρόσληψης τροφής. 
#Παρόλο που το ερωτηματολόγιο ΕΑΤ-26 δεν είναι διαγνωστικό εργαλείο για τις διαταραχές πρόσληψης τροφής, 
#έχει βρεθεί ότι είναι ιδιαίτερα αποτελεσματικό για την ανίχνευση πιθανών περιπτώσεων διαταραχής πρόσληψης τροφής.
eatingattitudes1 = retrieve_data('questionnaireData.xlsx','eatingattitudes')
eatingattitudes2 = retrieve_data('questionnaireData_ctrlpre.xlsx','eatingattitudes')
eatingattitudes = pd.concat([eatingattitudes1, eatingattitudes2], axis=0)
eatingattitudes.drop(['Unnamed: 0'],axis=1,inplace=True)
eatingattitudes.reset_index(drop=True, inplace=True)


ColumnNames = ["Score","pid"]
eatingattitudes = scores(eatingattitudes,ColumnNames)
characteristic = 'Κλίμακα στάσεων διατροφής (ΕΑΤ-26)'
number = 'Υψηλός κίνδυνος εμφάνισης διαταραχής πρόσληψης τροφής (σκορ>20): ' + str(len(eatingattitudes["Score"][eatingattitudes["Score"]>=20])) + '('+ str(round(100*len(eatingattitudes["Score"][eatingattitudes["Score"]>=20])/len(eatingattitudes["Score"]),2)) + '%)'   
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)
#---------------------------------------------------------------------------------------------------------------------

#------------------------------Strenghts and difficulties SDQ----------------------------------------------------------------------------------------#
#Tot: total, Emo: Emotional symptoms, Cond: Conduct problems, Hyper: Hyperactivity/inattention, Peer: Peer relationships problem, Beh: Prosocial behaviour 
#                             Normal         Bordeline       Abormal
#Total Difficulties Score      0-15             16-19         20-40
#Emotional Symptoms Score      0-5                6           7-10 
#Conduct problems Score        0-3                4           5-10  
#Hyperactivity/inattention     0-5                6           7-10 
#Peer problems                 0-3               4-5          6-10
#Prosocial Behaviour Score     6-10               5           0-4 
sdq1 = retrieve_data('questionnaireData.xlsx','strengthsdifficulties')
sdq2 = retrieve_data('questionnaireData_ctrlpre.xlsx','strengthsdifficulties')
sdq = pd.concat([sdq1, sdq2], axis=0)
sdq.drop(['Unnamed: 0'],axis=1,inplace=True)
sdq.reset_index(drop=True, inplace=True)

ColumnNames = ["Tot","Emo","Cond","Hyper","Peer","Beh","id"]
sdq = scores(sdq,ColumnNames)

def sdq_statistics(SDQ_name):
    Cat = ['Φυσιολογικό: ','Οριακό: ','Παθολογικό: ']
    l = 0
    for i in ['Nor','Bor','Abn']:
        if i=='Nor':
            temp = sdq[SDQ_name].str.find(i)
            number = Cat[l]  + str(len(temp[temp>-1])) + '(' + str(round(100*len(temp[temp>-1])/len(temp),2)) + '%)'
        else:
            temp = sdq[SDQ_name].str.find(i)
            number = number + ', ' + Cat[l] + str(len(temp[temp>-1])) + '(' + str(round(100*len(temp[temp>-1])/len(temp),2)) + '%)'
        l=l+1
    return number

characteristic = 'Δυνατότητες και δυσκολίες-Συνολικό'
number = sdq_statistics("Tot")
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Δυνατότητες και δυσκολίες-Συναισθηματικές διαταραχές'
number = sdq_statistics("Emo")
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Δυνατότητες και δυσκολίες-Προβλήματα συμπεριφοράς'
number = sdq_statistics("Cond")
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Δυνατότητες και δυσκολίες-Υπερκινητικότητα/διάσπαση προσοχής'
number = sdq_statistics("Hyper")
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Δυνατότητες και δυσκολίες-Προβλήματα ομηλίκων'
number = sdq_statistics("Peer")
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Δυνατότητες και δυσκολίες-Κοινωνική συμπεριφορά'
number = sdq_statistics("Beh")
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)
#-------------------------------------------------------------------------------------------------------------------------

#------------------------------------------Goneikos typos miteras---------------------------------------------------------
mother1 = retrieve_data('questionnaireData.xlsx','mother')
mother2 = retrieve_data('questionnaireData_ctrlpre.xlsx','mother')
mother = pd.concat([mother1, mother2], axis=0)
mother.drop(['Unnamed: 0'],axis=1,inplace=True)
mother.reset_index(drop=True, inplace=True)

ColumnNames = ["Ypostiriktiki","Afstiri","Epitreptiki","Aftarhiki","id"]
mother = scores(mother,ColumnNames)
characteristic = 'Γονεϊκός τύπος'
number = generate_descriptive_statistics(mother[["Ypostiriktiki","Afstiri","Epitreptiki","Aftarhiki"]],['Υποστηρικτική','Αυστηρή','Επιτρεπτική','Αυταρχική'],'continous',[])
dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)
#---------------------------------------------------------------------------------------------------------------------------

#----------------------------------------------------------Diet_behaviour---------------------------------------------------------------  
#CEBQ: Πενταβάθμια κλίμακα Likert (ποτέ, σπάνια, μερικές φορές, συχνά, πάντα)
#diet_behaviour = retrieve_data('foodhabits')
#print(diet_behaviour.columns)
#ColumnNames = ["Res","Sat","Fus","id"]#Xamilo Food Responsiveness, Ypsilo Satiety Responsiveness, Xamilo Food Fussiness 
#CEBQ = scores(diet_behaviour,ColumnNames)
#diet_behaviour = CEBQ.join(diet_behaviour[["ΑΡΙΘΜΟΣ ΚΥΡΙΩΝ ΓΕΥΜΑΤΩΝ - v",
#                                           "ΑΡΙΘΜΟΣ ΜΙΚΡΟΓΕΥΜΑΤΩΝ / ΣΝΑΚ - v",
#                                           "ΜΕΣΟΔΙΑΣΤΗΜΑΤΑ ΜΕΤΑΞΥ ΓΕΥΜΑΤΩΝ: - v",
#                                           "ΟΙΚΟΓΕΝΕΙΑΚΑ ΓΕΥΜΑΤΑ - v",
#                                           "ΠΡΟΣΩΠΑ ΦΡΟΝΤΙΔΑΣ  ΣΙΤΙΣΗΣ ΠΑΙΔΙΟΥ: - v", 
#                                           "ΡΥΘΜΟΣ ΣΙΤΙΣΗΣ - v",
#                                           "ΣΗΜΕΙΩΣΕΙΣ - v"]])
#characteristic = 'Διατροφική συμπεριφορά του παιδιού/CEBQ'
#number = generate_descriptive_statistics(diet_behaviour[["Res","Sat","Fus"]],
#                ['Απόκριση σε ερεθίσματα τροφής','Ρύθμιση πρόσληψης τροφής ανάλογα με το αίσθημα κορεσμού',
#                 'Ιδιότροπη συμπεριφορά με το φαγητό'],'continous',[])
#dsscores = dsscores.append({C:characteristic,N:number},ignore_index=True)

#Check if it is meaningful to include and the other variables

#--------------------------------------------------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
dsscores

,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Γονεϊκές πρακτικές σίτισης,"Γονείς-πρότυπα: 4.07 ± 0.79, Καταναγκαστικός έ..."
1,Σκορ κατάθλιψης,6.31±5.9
2,Κατηγορία σκορ κατάθλιψης,"Κανένα σύμπτωμα: 27(56.25)%, Ελάχιστα συμπτώμα..."
3,Κλίμακα στάσεων διατροφής (ΕΑΤ-26),Υψηλός κίνδυνος εμφάνισης διαταραχής πρόσληψης...
4,Δυνατότητες και δυσκολίες-Συνολικό,"Φυσιολογικό: 35(71.43%), Οριακό: 3(6.12%), Παθ..."
5,Δυνατότητες και δυσκολίες-Συναισθηματικές διατ...,"Φυσιολογικό: 33(67.35%), Οριακό: 3(6.12%), Παθ..."
6,Δυνατότητες και δυσκολίες-Προβλήματα συμπεριφοράς,"Φυσιολογικό: 27(55.1%), Οριακό: 11(22.45%), Πα..."
7,Δυνατότητες και δυσκολίες-Υπερκινητικότητα/διά...,"Φυσιολογικό: 38(77.55%), Οριακό: 5(10.2%), Παθ..."
8,Δυνατότητες και δυσκολίες-Προβλήματα ομηλίκων,"Φυσιολογικό: 39(79.59%), Οριακό: 4(8.16%), Παθ..."
9,Δυνατότητες και δυσκολίες-Κοινωνική συμπεριφορά,"Φυσιολογικό: 49(100.0%), Οριακό: 0(0.0%), Παθο..."


In [ ]:
#Somatometric-Physical Examination
#------------------------------------------------Somatometrika - physical examination----------------------------------------------------------- 
somatometric1 = retrieve_data('questionnaireData.xlsx','medial_clinical')
somatometric2 = retrieve_data('questionnaireData_ctrlpre.xlsx','medial_clinical')
somatometric = pd.concat([somatometric1, somatometric2], axis=0)
somatometric.drop(['Unnamed: 0'],axis=1,inplace=True)
somatometric.reset_index(drop=True, inplace=True)

somatometric = somatometric[["BMI - v",
                             "Tanner Pubis - v",
                             "ΑΡΤΗΡΙΑΚΗ ΠΙΕΣΗ ΔΙΑΣΤΟΛΙΚΗ - v",
                             "ΑΡΤΗΡΙΑΚΗ ΠΙΕΣΗ ΣΥΣΤΟΛΙΚΗ - v",
                             "Βάρος - v",
                             "Ειδικά Χαρακτηριστικά - v",
                             "Μελανίζουσα ακάνθωση - v", #0-όχι,1-Τράχηλος,2-μασχαλιαίες χώρες,3-μηροβουβωνικές 
                             "ΠΕΡΙΜΕΤΡΟΣ ΜΕΣΗΣ - v",
                             "ΥΨΟΣ - v",
                             "Tanner Breast - v",
                             "Tanner μασχάλη - v",
                             "Θυρεοειδής - v"]]#Θυροειδής-1: αψηλάφητος
somatometric["Βάρος - v"] = somatometric["Βάρος - v"].replace(to_replace=',', value='.', regex=True)

characteristic = 'ΔΜΣ'
number = generate_descriptive_statistics(somatometric["BMI - v"],['Τεστ'],'continous',[])
dssoma = dssoma.append({C:characteristic,N:number},ignore_index=True)
characteristic = 'Βάρος'
number = generate_descriptive_statistics(somatometric["Βάρος - v"],['Τεστ'],'continous',[])
dssoma = dssoma.append({C:characteristic,N:number},ignore_index=True)
characteristic = 'Ύψος'
number = generate_descriptive_statistics(somatometric["ΥΨΟΣ - v"],['Τεστ'],'continous',[])
dssoma = dssoma.append({C:characteristic,N:number},ignore_index=True)
characteristic = 'Περίμετρος μέσης'
number = generate_descriptive_statistics(somatometric["ΠΕΡΙΜΕΤΡΟΣ ΜΕΣΗΣ - v"],['Τεστ'],'continous',[])
dssoma = dssoma.append({C:characteristic,N:number},ignore_index=True)
characteristic = 'Tanner Bubis'
number = generate_descriptive_statistics(somatometric["Tanner Pubis - v"],['Stage I','Stage II','Stage III','Stage IV','Stage V']
                                         ,'categorical2',[1,2,3,4,5])
dssoma = dssoma.append({C:characteristic,N:number},ignore_index=True)
characteristic = 'Αρτηριακή πίεση (διαστολική)'
number = generate_descriptive_statistics(somatometric["ΑΡΤΗΡΙΑΚΗ ΠΙΕΣΗ ΔΙΑΣΤΟΛΙΚΗ - v"],['τεστ'] ,'continous',[])
dssoma = dssoma.append({C:characteristic,N:number},ignore_index=True)
characteristic = 'Αρτηριακή πίεση (συστολική)'
number = generate_descriptive_statistics(somatometric["ΑΡΤΗΡΙΑΚΗ ΠΙΕΣΗ ΣΥΣΤΟΛΙΚΗ - v"],['τεστ'] ,'continous',[])
dssoma = dssoma.append({C:characteristic,N:number},ignore_index=True)
characteristic = 'Ειδικά χαρακτηριστικά'
for i in somatometric["Ειδικά Χαρακτηριστικά - v"].unique():
    if i==somatometric["Ειδικά Χαρακτηριστικά - v"].unique()[0]:
        number = i + ':' + str(len(somatometric["Ειδικά Χαρακτηριστικά - v"][somatometric["Ειδικά Χαρακτηριστικά - v"]==i])) 
    else:
        number = number + ', ' + i + ':' + str(len(somatometric["Ειδικά Χαρακτηριστικά - v"][somatometric["Ειδικά Χαρακτηριστικά - v"]==i])) 
dssoma = dssoma.append({C:characteristic,N:number},ignore_index=True)
characteristic = 'Μελανίζουσα ακάνθωση'
temp = somatometric["Μελανίζουσα ακάνθωση - v"]
cate = ['Όχι','Τράχηλος','Μασχαλιαίες χώρες','Μηροβουβωνικές περιοχές']
l=0
for i in ['0','1','2','3']:
    if i=='0':
        number = cate[l] + ': ' + str(len(temp.astype(str).str.find(i)[temp.astype(str).str.find(i)>-1])) + '(' + str(round(100*len(temp.astype(str).str.find(i)[temp.astype(str).str.find(i)>-1])/len(temp),2)) + '%)'
    else:
        number = number + ', ' + cate[l] + ': ' + str(len(temp.astype(str).str.find(i)[temp.astype(str).str.find(i)>-1])) + '(' + str(round(100*len(temp.astype(str).str.find(i)[temp.astype(str).str.find(i)>-1])/len(temp),2)) + '%)'
    l=l+1
dssoma = dssoma.append({C:characteristic,N:number},ignore_index=True)
#-------------------------------------------------------------------------------------------------------------


In [ ]:
dssoma

,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,ΔΜΣ,31.7±6.93
1,Βάρος,74.25±24.06
2,Ύψος,151.27±13.11
3,Περίμετρος μέσης,93.83±24.19
4,Tanner Bubis,"Stage I: 12(40.0%), Stage II: 8(26.67%), Stage..."
5,Αρτηριακή πίεση (διαστολική),74.33±8.2
6,Αρτηριακή πίεση (συστολική),114.1±11.29
7,Ειδικά χαρακτηριστικά,"Ανασπώμενος δεξις όρχις:1, -:14, Ύβος, ραγάδε..."
8,Μελανίζουσα ακάνθωση,"Όχι: 14(46.67%), Τράχηλος: 14(46.67%), Μασχαλι..."


In [ ]:
#Biochemical
dsbiochemical = pd.DataFrame()
#----------------------------------------Biohimikoi deiktes---------------------------------------------------------
biochem1 = retrieve_data('questionnaireData.xlsx','medical_biochem')
biochem2 = retrieve_data('questionnaireData_ctrlpre.xlsx','medical_biochem')
biochem = pd.concat([biochem1, biochem2], axis=0)
biochem.drop(['Unnamed: 0'],axis=1,inplace=True)
biochem.reset_index(drop=True, inplace=True)

biochem = biochem[["pid",
                  "25 (OH) vitamin D - v",
                  "ACTH - v",
                  "Chol - v",
                  "FSH - v",
                  "FreeT4 - v",
                  "HDL - v",
                  "HbA1c - v",
                  "LDL - v",
                  "LH - v",
                  "Lp(a) - v",
                  "SGOT - v",
                  "SGPT - v",
                  "TG - v",
                  "TSH - v",
                  "Γλυκόζη - v",
                  "Ινσουλίνη - v",
                  "Κορτιζόλη - v",
                  "Οιστραδιόλη - v",
                  "Τεστοστερόνη - v"]]

biochem = biochem.replace(to_replace=',', value='.', regex=True)

def biochem_statistics(name,dsbiochemical,characteristic):
    series_is_string = pd.api.types.is_string_dtype(biochem[name])
    if series_is_string==True:
        temp = biochem[name].str.find('-')
        temp1 = temp[temp>-1]
        print(temp1)
        temp2 = biochem[name].drop(temp1.index)
        number_missing = 'Ελλείψεις: '+ str(len(temp1)) + '(' + str(round(100*len(temp1)/len(temp),2)) +'%)'
        check1 = temp2.str.find('>')
        check2 = temp2.str.find('<')
    else:
        temp2 =  biochem[name]
        number_missing = 'Ελλείψεις: 0'
        check1=pd.DataFrame({'Test':[],'Test':[]})
        check2=pd.DataFrame({'Test':[],'Test':[]})
    if len(check1[check1>-1])==0 and len(check2[check2>-1])==0: 
        number = number_missing + ', ' + generate_descriptive_statistics(temp2,['test'],'continous',[])
    elif len(check1[check1>-1])>0:
        check = check1 
        temp4 = temp2.drop(check[check>-1].index)
        temp5 = temp2.loc[check[check>-1].index]
        temp6 = temp5.str.split('>',n=1,expand = True)#.value_counts()
        temp7 = temp6.iloc[:,1].replace(to_replace=' ', value='', regex=True)
        temp7 = pd.to_numeric(temp7)
        unique_values = temp7.unique()
        number_above = 'Υψηλότερη από '
        for i in range(len(unique_values)):
            s_t = temp7[temp7==unique_values[i]]
            s = len(s_t)
            if i==0:
                number_above = number_above + str(unique_values[i]) +  ': ' + str(s) + '(' + str(round(100*s/len(temp4),2)) + '%)'
            else:
                number_above = number_above + str(unique_values[i]) +  ': ' + str(s) + '(' + str(round(100*s/len(temp4),2)) + '%)'
        number = number_missing + ', ' + number_above + ', ' + generate_descriptive_statistics(temp4,['test'],'continous',[])
    elif len(check2[check2>-1])>0:
        check = check2
        temp4 = temp2.drop(check[check>-1].index)
        temp5 = temp2.loc[check[check>-1].index]
        temp6 = temp5.str.split('<',n=1,expand = True)#.value_counts()
        temp7 = temp6.iloc[:,1].replace(to_replace=' ', value='', regex=True)
        temp7 = pd.to_numeric(temp7)
        unique_values = temp7.unique()
        number_above = 'Χαμηλότερη από '
        for i in range(len(unique_values)):
            s_t = temp7[temp7==unique_values[i]]
            s = len(s_t)
            if i==0:
                number_above = number_above + str(unique_values[i]) +  ': ' + str(s) + '(' + str(round(100*s/len(temp4),2)) + '%)'
            else:
                number_above = number_above + str(unique_values[i]) +  ': ' + str(s) + '(' + str(round(100*s/len(temp4),2)) + '%)'
        number = number_missing + ', ' + number_above + ', ' + generate_descriptive_statistics(temp4,['test'],'continous',[])
    dsbiochemical = dsbiochemical.append({C:characteristic,N:number},ignore_index=True)
    return dsbiochemical

name = "25 (OH) vitamin D - v"
characteristic = '25-υδροξυβιταμίνη D (ng/mL)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "ACTH - v"
characteristic = 'Αδρενοκορτικοτρόπος ορμόνη (pg/mL)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "Chol - v"
characteristic = 'Χοληστερόλη (mg/dl)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "FSH - v"
characteristic = 'Θυλακιοτρόπος Ορμόνη (mIU/ml)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "FreeT4 - v"
characteristic = 'Ελεύθερη θυροξίνη (ng/dL)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "HDL - v"
characteristic = 'HDL χοληστερίνη (mg/dl)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "HbA1c - v"
characteristic = 'Γλυκοζυλιωμένη αιμοσφαιρίνη (mg/dl)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "LDL - v"
characteristic = 'LDL χοληστερίνη (mg/dl)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "LH - v"
characteristic = 'Ωχρινοτρόπος Ορμόνη (mIU/ml)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "Lp(a) - v"
characteristic = 'Λιποπρωτεΐνη Α (mIU/ml)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "SGOT - v"
characteristic = 'Oξαλοξική τρανσαμινάση (U/L)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "SGPT - v"
characteristic = 'Πυροσταφιλική τρανσαμινάση (U/L)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "TG - v"
characteristic = 'Θυρεοσφαιρίνη (U/L)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "TSH - v"
characteristic = 'Θυρεοτρόπος Ορμόνη (mIU/L)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "Ινσουλίνη - v"
characteristic = 'Ινσουλίνη (μIU/mL)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "Γλυκόζη - v"
characteristic = 'Γλυκόζη (mg/dL)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

name = "Κορτιζόλη - v"
characteristic = 'Κορτιζόλη (μg/dL)'
dsbiochemical = biochem_statistics(name,dsbiochemical,characteristic)

characteristic = 'Οιστραδιόλη (pg/ml)'
temp = biochem["Οιστραδιόλη - v"].str.find('-')
temp2 = temp[temp>-1] 
temp3 = biochem["Οιστραδιόλη - v"].drop(temp2.index)
temp = temp3.str.find('<')
number = 'Φυσιολογικές τιμές: ' + str(len(temp[temp>-1])) + '(' + str(round(100*len(temp[temp>-1])/len(temp),2)) + '%)'
dsbiochemical = dsbiochemical.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Τεστοστερόνη (ng/dL)'
temp = biochem["Οιστραδιόλη - v"].str.find('-')
temp2 = temp[temp>-1] 
temp3 = biochem["Οιστραδιόλη - v"].drop(temp2.index)
temp = temp3.str.find('<')
number = 'Φυσιολογικές τιμές: ' + str(len(temp[temp>-1])) + '(' + str(round(100*len(temp[temp>-1])/len(temp),2)) + '%)'
dsbiochemical = dsbiochemical.append({C:characteristic,N:number},ignore_index=True)
dsbiochemical
#---------------------------------------------------------------------------------------------------------------------------------

Series([], Name: 25 (OH) vitamin D - v, dtype: int64)
2     0
3     0
20    0
38    0
42    0
Name: ACTH - v, dtype: int64
24    0.0
Name: Chol - v, dtype: float64
2     0
20    0
Name: FSH - v, dtype: int64
10    0
20    0
Name: FreeT4 - v, dtype: int64
24    0.0
Name: HDL - v, dtype: float64
0     0
3     0
19    0
34    0
42    0
Name: HbA1c - v, dtype: int64
24    0.0
Name: LDL - v, dtype: float64
2     0
20    0
Name: LH - v, dtype: int64
3     0
20    0
24    0
31    0
38    0
42    0
Name: Lp(a) - v, dtype: int64
Series([], Name: SGPT - v, dtype: float64)
24    0.0
Name: TG - v, dtype: float64
10    0
Name: TSH - v, dtype: int64
3     0
14    0
42    0
Name: Ινσουλίνη - v, dtype: int64
36    0.0
Name: Γλυκόζη - v, dtype: float64
3     0
20    0
38    0
Name: Κορτιζόλη - v, dtype: int64


,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,25-υδροξυβιταμίνη D (ng/mL),"Ελλείψεις: 0(0.0%), 20.46±5.57"
1,Αδρενοκορτικοτρόπος ορμόνη (pg/mL),"Ελλείψεις: 5(10.42%), 36.87±53.9"
2,Χοληστερόλη (mg/dl),"Ελλείψεις: 1(2.08%), 158.67±25.04"
3,Θυλακιοτρόπος Ορμόνη (mIU/ml),"Ελλείψεις: 2(4.17%), 2.24±1.82"
4,Ελεύθερη θυροξίνη (ng/dL),"Ελλείψεις: 2(4.17%), 5.59±6.18"
5,HDL χοληστερίνη (mg/dl),"Ελλείψεις: 1(2.08%), 49.37±10.93"
6,Γλυκοζυλιωμένη αιμοσφαιρίνη (mg/dl),"Ελλείψεις: 5(10.42%), 5.46±0.29"
7,LDL χοληστερίνη (mg/dl),"Ελλείψεις: 1(2.08%), 89.0±21.1"
8,Ωχρινοτρόπος Ορμόνη (mIU/ml),"Ελλείψεις: 2(4.17%), Χαμηλότερη από 0.1: 13(44..."
9,Λιποπρωτεΐνη Α (mIU/ml),"Ελλείψεις: 6(12.5%), Χαμηλότερη από 5.71: 1(2...."


In [ ]:
#Atomiko-Progennhtiko-Perigennhtiko Istoriko
dshistory_atomiko = pd.DataFrame()
ds_history_progenhtiko = pd.DataFrame()
ds_history_perigennhtiko = pd.DataFrame()
#----------------------------------------Atomiko Anamnistiko---------------------------------------------
medical_history1 = retrieve_data('questionnaireData.xlsx','medical_personal')
medical_history2 = retrieve_data('questionnaireData_ctrlpre.xlsx','medical_personal')
medical_history = pd.concat([medical_history1, medical_history2], axis=0)
medical_history.drop(['Unnamed: 0'],axis=1,inplace=True)
medical_history.reset_index(drop=True, inplace=True)

medical_history = medical_history[["pid",
                                   "Άλλα νοσήματα - v",
                                   "ΑΛΛΕΡΓΙΕΣ - v",
                                   "ΔΙΑΡΚΕΙΑ  θΗΛΑΣΜΟΥ ΣΕ ΜΗΝΕΣ - v",
                                   "ΔΙΑΡΚΕΙΑ ΑΠΟΚΛΕΙΣΤΙΚΟΥ ΘΗΛΑΣΜΟΥ (ΜΗΝΕΣ): - v",
                                   "ΔΙΑΡΚΕΙΑ ΜΙΚΤΟΥ ΘΗΛΑΣΜΟΥ (ΜΗΝΕΣ): - v",
                                   "ΘΗΛΑΣΜΟΣ - v",
                                   "ΝΟΣΗΛΕΙΕΣ - v", #0:OXI,1:Ουρολοίμωξη,2:Πνευμονία,3:Εμπύρετη νόσος, 4: Άλλα
                                   "Συγγενή νοσήματα - v",#0: όχι, 1: καρδιοπάθεια, 2: γενετικό νόσημα
                                   "ΦΑΡΜΑΚΑ / ΒΙΤΑΜΙΝΕΣ / ΣΥΜΠΛΗΡΩΜΑΤΑ - v",
                                   "Χρόνια νοσήματα - v",#0: όχι, 1: Άσθμα, 2: Αλλεργική ρινιτίδα, 3: Ατοπική δερματίτιδα, 3 (λαθος απο UBI): Χειρουργείο για Αδενοειδείς εκβλαστήσεις ή Αμυγδαλεκτομή 
                                   "Χρόνια νοσήματα - t",
                                   "Είδος ΦΑΡΜΑΚΑ / ΒΙΤΑΜΙΝΕΣ / ΣΥΜΠΛΗΡΩΜΑΤΑ - v",
                                   "Είδος Αλλεργίας - v",
                                   "Aλλα αίτια νοσηλείας - v"]]

characteristic = 'Άλλα νοσήματα'
temp1 = medical_history["Άλλα νοσήματα - v"].str.find("Οχι") 
temp2 = medical_history["Άλλα νοσήματα - v"].str.find("όχι")  
temp3 = medical_history["Άλλα νοσήματα - v"].str.find("οχι")  
temp = pd.concat([temp1[temp1>-1],temp2[temp2>-1],temp3[temp3>-1]])
no_diseases = len(medical_history["Άλλα νοσήματα - v"].loc[temp.index])
temp_d = medical_history["Άλλα νοσήματα - v"].drop(temp.index)
diseases = len(medical_history["Άλλα νοσήματα - v"].drop(temp.index))

l=1
for i in temp_d:
    if l==1:
        number_diseases = i
        l=l+1
    else:
        number_diseases = number_diseases + ', ' + i

number = 'Όχι: '+str(no_diseases) + '(' + str(round(100*no_diseases/(diseases+no_diseases),2)) +'%), ' + 'Άλλα νοσήματα: '+ number_diseases
dshistory_atomiko = dshistory_atomiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Αλλεργίες'
number = generate_descriptive_statistics(medical_history["ΑΛΛΕΡΓΙΕΣ - v"],['Όχι','Ναι'],'categorical2',[0,1])
dshistory_atomiko = dshistory_atomiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Θηλασμός'
number = generate_descriptive_statistics(medical_history["ΘΗΛΑΣΜΟΣ - v"],['Όχι','Ναι'],'categorical2',[0,1])
dshistory_atomiko = dshistory_atomiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Διάρκεια θηλασμού (Μήνες)'
number = generate_descriptive_statistics(medical_history["ΔΙΑΡΚΕΙΑ  θΗΛΑΣΜΟΥ ΣΕ ΜΗΝΕΣ - v"],['Test'],'continous',[0,1])
dshistory_atomiko = dshistory_atomiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Διάρκεια αποκλειστικού θηλασμού (Μήνες)'
number = generate_descriptive_statistics(medical_history["ΔΙΑΡΚΕΙΑ ΑΠΟΚΛΕΙΣΤΙΚΟΥ ΘΗΛΑΣΜΟΥ (ΜΗΝΕΣ): - v"],['Test'],'continous',[0,1])
dshistory_atomiko = dshistory_atomiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Διάρκεια μικτού θηλασμού (Μήνες)'
number = generate_descriptive_statistics(medical_history["ΔΙΑΡΚΕΙΑ ΜΙΚΤΟΥ ΘΗΛΑΣΜΟΥ (ΜΗΝΕΣ): - v"],['Test'],'continous',[0,1])
dshistory_atomiko = dshistory_atomiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Νοσηλείες'
number = generate_descriptive_statistics(medical_history["ΝΟΣΗΛΕΙΕΣ - v"],['Όχι','Ουρολοίμωξη','Πνευμονία','Εμπύρετη νόσος','Άλλα αίτια'],'categorical2',[0,1,2,3,4])
dshistory_atomiko = dshistory_atomiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Συγγενή νοσήματα'
number = generate_descriptive_statistics(medical_history["Συγγενή νοσήματα - v"],['Όχι','Καρδιοπάθεια','Γενετικό νόσημα'],'categorical2',[0,1,2])
dshistory_atomiko = dshistory_atomiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Χρόνια νοσήματα'
n = ['Όχι','Άσθμα','Αλλεργική ρινιτίδα','Ατοπική δερματίτιδα','Χειρουργείο για Αδενοειδείς εκβλαστήσεις ή Αμυγδαλεκτομή']
for i in n:
    temp = medical_history["Χρόνια νοσήματα - t"].str.find(i)
    if i=='Όχι':
        number = i + ': ' + str(len(temp[temp>-1])) + '(' + str(round(100*len(temp[temp>-1])/len(temp),2))+'%)'
    else:
        number = number + ', ' + i + ': ' + str(len(temp[temp>-1])) + '(' + str(round(100*len(temp[temp>-1])/len(temp),2)) +'%)'
dshistory_atomiko = dshistory_atomiko.append({C:characteristic,N:number},ignore_index=True)

#---------------------------------------------------------------------------------------------------------------------------------

#----------------------------------------Progennhtiko Anamnistiko---------------------------------------------
medical_prebirth1 = retrieve_data('questionnaireData.xlsx','medical_prebirth')
medical_prebirth2 = retrieve_data('questionnaireData_ctrlpre.xlsx','medical_prebirth')
medical_prebirth = pd.concat([medical_prebirth1, medical_prebirth2], axis=0)
medical_prebirth.drop(['Unnamed: 0'],axis=1,inplace=True)
medical_prebirth.reset_index(drop=True, inplace=True)

medical_prebirth = medical_prebirth[["pid",
                                    "IUGR (ενδομήτρια καθυστέρηση αύξησης) - v",
                                    "SGA - v",
                                    "ΑΛΛΑ ΠΡΟΒΛΗΜΑΤΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v",
                                    "ΒΑΡΟΣ ΜΗΤΕΡΑΣ ΠΡΟ ΚΥΗΣΗΣ - v",
                                    "Εβδομάδες κύησης - v",
                                    "ΗΜΕΡΟΜΗΝΙΑ ΓΕΝΝΗΣΗΣ - v",
                                    "ΗΜΕΡΟΜΗΝΙΑ ΚΛΙΝΙΚΗΣ ΕΞΕΤΑΣΗΣ - v",
                                    "Κ.Τ ΛΟΓΩ - v",
                                    "ΚΑΠΝΙΣΜΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v",
                                    "ΚΑΠΝΙΣΜΑ ΠΡΟ ΚΥΗΣΗΣ - v",
                                    "ΚΥΗΣΗ - v",
                                    "ΠΡΟΣΛΗΨΗ ΒΑΡΟΥΣ ΚΑΤΑ ΤΗΝ ΕΓΚΥΜΟΣΥΝΗ - v",
                                    "Σ.Δ. ΚΥΗΣΗΣ - v",
                                    "ΤΟΚΕΤΟΣ - v",
                                    "βάρος γέννησης - v",
                                    "μήκος σώματος - v",
                                    "περίμετρος κεφαλής - v"]]

characteristic = 'Ενδομήτρια καθυστέρηση της ανάπτυξης'
number = generate_descriptive_statistics(medical_prebirth["IUGR (ενδομήτρια καθυστέρηση αύξησης) - v"],['Όχι','Ναι'],'categorical2',[0,1])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Aνεπαρκή ανάπτυξη για την ηλικία κύησης'
number = generate_descriptive_statistics(medical_prebirth["SGA - v"],['Όχι','Ναι'],'categorical2',[0,1])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Άλλα προβλήματα κατά την κύηση'
temp1 = medical_prebirth["ΑΛΛΑ ΠΡΟΒΛΗΜΑΤΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v"].str.find('οχι')
temp2 = medical_prebirth["ΑΛΛΑ ΠΡΟΒΛΗΜΑΤΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v"].str.find('Οχι')
no = len(temp1[temp1>-1])+len(temp2[temp2>-1])
ind = [temp1[temp1>-1].index,temp2[temp2>-1].index]
temp = medical_prebirth["ΑΛΛΑ ΠΡΟΒΛΗΜΑΤΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v"].drop(temp1[temp1>-1].index)
temp = temp.drop(temp2[temp2>-1].index)
no_number = 'Όχι: ' + str(no) + '(' + str(round(100*no/(len(temp1)),2))
for i in temp:
    if i == temp.iloc[0]:
        yes_number = temp.iloc[0]
    else:
        yes_number = yes_number + ', '+ i 
number = no_number + ', ' + yes_number
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Βάρος μητέρας προ κύησης'
number = generate_descriptive_statistics(medical_prebirth["ΒΑΡΟΣ ΜΗΤΕΡΑΣ ΠΡΟ ΚΥΗΣΗΣ - v"],['Όχι'],'continous',[0,1])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Εβδομάδες κύησης'
number = generate_descriptive_statistics(medical_prebirth["Εβδομάδες κύησης - v"],['Όχι'],'continous',[0,1])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Κάπνισμα κατά την κύηση'
number = generate_descriptive_statistics(medical_prebirth["ΚΑΠΝΙΣΜΑ ΚΑΤΑ ΤΗΝ ΚΥΗΣΗ - v"],['Όχι','Ναι'],'categorical2',[0,1])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Κάπνισμα προ κύησης'
number = generate_descriptive_statistics(medical_prebirth["ΚΑΠΝΙΣΜΑ ΠΡΟ ΚΥΗΣΗΣ - v"],['Όχι','Ναι'],'categorical2',[0,1])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Κύηση'
number = generate_descriptive_statistics(medical_prebirth["ΚΥΗΣΗ - v"],['Τελειόμηνη (>37 βδ.)','Πρόωρη (<37 βδ.)'],'categorical2',[1,2])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πρόσληψη βάρους κατά την εγκυμοσύνη'
number = generate_descriptive_statistics(medical_prebirth["ΠΡΟΣΛΗΨΗ ΒΑΡΟΥΣ ΚΑΤΑ ΤΗΝ ΕΓΚΥΜΟΣΥΝΗ - v"],['Test'],'continous',[1,2])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Σακχαρώδης Διαβήτης Κύησης'
number = number = generate_descriptive_statistics(medical_prebirth["Σ.Δ. ΚΥΗΣΗΣ - v"],['Όχι','Ινσουλινοθεραπεία','Δίαιτα'],'categorical2',[0,1,2])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Τοκετός'
number = number = generate_descriptive_statistics(medical_prebirth["ΤΟΚΕΤΟΣ - v"],['Φ.Τ','Κ.Τ'],'categorical2',[0,1])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Βάρος γέννησης'
number = generate_descriptive_statistics(medical_prebirth["βάρος γέννησης - v"],['Test'],'continous',[1,2])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Μήκος σώματος'
number = generate_descriptive_statistics(medical_prebirth["μήκος σώματος - v"],['Test'],'continous',[1,2])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Περίμετρος κεφαλής'
number = generate_descriptive_statistics(medical_prebirth["περίμετρος κεφαλής - v"],['Test'],'continous',[1,2])
ds_history_progenhtiko = ds_history_progenhtiko.append({C:characteristic,N:number},ignore_index=True)

#----------------------------------------Perigennhtiko Istoriko---------------------------------------------
medical_birth1 = retrieve_data('questionnaireData.xlsx','medical_birth')
medical_birth2 = retrieve_data('questionnaireData_ctrlpre.xlsx','medical_birth')
medical_birth = pd.concat([medical_birth1, medical_birth2], axis=0)
medical_birth.drop(['Unnamed: 0'],axis=1,inplace=True)
medical_birth.reset_index(drop=True, inplace=True)

medical_birth = medical_birth[["pid",
                                  "Αίτιο - v",
                                  "ΗΜΕΡΕΣ ΠΑΡΑΜΟΝΗΣ - v",
                                  "ΙΚΤΕΡΟΣ - v",
                                  "ΛΟΙΠΑ ΠΡΟΒΛΗΜΑΤΑ - v",
                                  "ΠΑΡΑΜΟΝΗ ΣΕ ΜΕΝ / ΜΑΦ - v",
                                  "ΥΠΟΓΛΥΚΑΙΜΙΑ - v"]]

temp = pd.notnull(medical_birth["ΗΜΕΡΕΣ ΠΑΡΑΜΟΝΗΣ - v"])
temp2 = medical_birth["ΗΜΕΡΕΣ ΠΑΡΑΜΟΝΗΣ - v"]
hospital = len(temp[temp==True])

characteristic = 'Νοσηλεία κατά την γέννηση'
number = 'Νοσηλείες: ' + str(hospital) + '(' + str(round(100*hospital/len(temp),2)) + '%)' + ', Ημέρες Παραμονής: '+str(round(temp2[temp==True].mean(),2)) + '±' + str(round(temp2[temp==True].std(),2))
ds_history_perigennhtiko = ds_history_perigennhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Ίκτερος'
number = number = generate_descriptive_statistics(medical_birth["ΙΚΤΕΡΟΣ - v"],['Όχι','Ναι'],'categorical2',[0,1])
ds_history_perigennhtiko = ds_history_perigennhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Παραμονή σε Μονάδα Εντατικής Θεραπείας/Μονάδα Αυξημένης Φροντίδας'
number = number = generate_descriptive_statistics(medical_birth["ΠΑΡΑΜΟΝΗ ΣΕ ΜΕΝ / ΜΑΦ - v"],['Όχι','Ναι'],'categorical2',[0,1])
ds_history_perigennhtiko = ds_history_perigennhtiko.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Υπογλυκαιμία'
number = number = generate_descriptive_statistics(medical_birth["ΥΠΟΓΛΥΚΑΙΜΙΑ - v"],['Όχι','Ναι'],'categorical2',[0,1])
ds_history_perigennhtiko = ds_history_perigennhtiko.append({C:characteristic,N:number},ignore_index=True)

In [ ]:
dshistory_atomiko

,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Άλλα νοσήματα,"Όχι: 13(27.08%), Άλλα νοσήματα: Δίπτυχη αορτικ..."
1,Αλλεργίες,"Όχι: 41(85.42%), Ναι: 7(14.58%)"
2,Θηλασμός,"Όχι: 14(29.17%), Ναι: 34(70.83%)"
3,Διάρκεια θηλασμού (Μήνες),8.51±13.66
4,Διάρκεια αποκλειστικού θηλασμού (Μήνες),2.94±5.03
5,Διάρκεια μικτού θηλασμού (Μήνες),5.74±13.11
6,Νοσηλείες,"Όχι: 31(64.58%), Ουρολοίμωξη: 2(4.17%), Πνευμο..."
7,Συγγενή νοσήματα,"Όχι: 47(97.92%), Καρδιοπάθεια: 1(2.08%), Γενετ..."
8,Χρόνια νοσήματα,"Όχι: 40(83.33%), Άσθμα: 2(4.17%), Αλλεργική ρι..."


In [ ]:
ds_history_perigennhtiko

,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Νοσηλεία κατά την γέννηση,"Νοσηλείες: 12(25.0%), Ημέρες Παραμονής: 17.92±..."
1,Ίκτερος,"Όχι: 39(81.25%), Ναι: 9(18.75%)"
2,Παραμονή σε Μονάδα Εντατικής Θεραπείας/Μονάδα ...,"Όχι: 36(75.0%), Ναι: 12(25.0%)"
3,Υπογλυκαιμία,"Όχι: 48(100.0%), Ναι: 0(0.0%)"


In [ ]:
ds_history_progenhtiko

,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Ενδομήτρια καθυστέρηση της ανάπτυξης,"Όχι: 47(97.92%), Ναι: 1(2.08%)"
1,Aνεπαρκή ανάπτυξη για την ηλικία κύησης,"Όχι: 46(95.83%), Ναι: 2(4.17%)"
2,Άλλα προβλήματα κατά την κύηση,"Όχι: 16(33.33, -"
3,Βάρος μητέρας προ κύησης,70.48±13.58
4,Εβδομάδες κύησης,38.04±2.06
5,Κάπνισμα κατά την κύηση,"Όχι: 35(72.92%), Ναι: 13(27.08%)"
6,Κάπνισμα προ κύησης,"Όχι: 28(58.33%), Ναι: 20(41.67%)"
7,Κύηση,"Τελειόμηνη (>37 βδ.): 42(87.5%), Πρόωρη (<37 β..."
8,Πρόσληψη βάρους κατά την εγκυμοσύνη,19.27±18.11
9,Σακχαρώδης Διαβήτης Κύησης,"Όχι: 44(91.67%), Ινσουλινοθεραπεία: 2(4.17%), ..."


In [ ]:
#----------------------------------------Food habits pilot---------------------------------------------
food_habits_plt = retrieve_data('questionnaireData.xlsx','foodhabits')
#food_habits_pre = retrieve_data('questionnaireData_ctrlpre.xlsx','foodhabits')
#food_habits = pd.concat([food_habits1, food_habits2], axis=0)
#food_habits.drop(['Unnamed: 0'],axis=1,inplace=True)
#food_habits.reset_index(drop=True, inplace=True)

food_habits_plt = food_habits_plt[["pid",
                           "ΜΕΣΟΔΙΑΣΤΗΜΑΤΑ ΜΕΤΑΞΥ ΓΕΥΜΑΤΩΝ: - v",
                           "ΑΡΙΘΜΟΣ ΜΙΚΡΟΓΕΥΜΑΤΩΝ / ΣΝΑΚ - v",
                           "Πόσο συχνά τρώει το παιδί σας ψάρια ή θαλασσινά; - v",
                           "ΠΡΟΣΩΠΑ ΦΡΟΝΤΙΔΑΣ  ΣΙΤΙΣΗΣ ΠΑΙΔΙΟΥ: - v",
                           "Εάν έφαγε ψάρι ή θαλασσινό το παιδί σας τον προηγούμενο μήνα προσδιορίστε το είδος: - v",
                           "Πόσο συχνά χρησιμοποιείτε ελαιόλαδο στο μαγείρεμα; - v",
                           "Το παιδί μου ζητάει συνεχώς φαγητό - v",
                           "Πόσο συχνά τρώει το παιδί σας όσπρια; - v",
                           "Πόσο συχνά χρησιμοποιείτε βούτυρο; - v",
                           "Ακόμη κι αν έχει χορτάσει, το παιδί μου βρίσκει χώρο για το αγαπημένο του φαγητό - v",
                           "Πόσο συχνά χρησιμοποιείτε άλλα φυτικά έλαια στο μαγείρεμα; - v",
                           "Εάν είχε την επιλογή, το παιδί μου θα έτρωγε την περισσότερη ώρα της ημέρας - v",
                           "Πόσο συχνά τρώει το παιδί σας ξηρούς καρπούς (μια μερίδα αντιστοιχεί σε μια χούφτα ξηρών καρπών (π.χ. 18 αμύγδαλα, 6 ολόκληρα καρύδια, 3 κουταλιές της σούπας ηλιόσπορους)  ή προϊόντα τους  όπως ταχίνι, φυστικοβούτυρο κ.ά. (μια μερίδα αντιστοιχεί σε 1½  κουταλιά της σούπας των 15ml ή 25γρ. π.χ. ταχίνι) - v",
                           "Πόσο συχνά χρησιμοποιείτε μαργαρίνη; - v",                          
                           "Πόσες μερίδες φρούτων καταναλώνει το παιδί σας (μέσος όρος); - v",     
                           "Πόσες μερίδες λαχανικών (ωμών ή βραστών ή στον ατμό ή ψητών) καταναλώνει το παιδί σας (μέσος όρος); - v",
                           "Πόσες μερίδες γαλακτοκομικών (ένα ποτήρι γάλα των 250ml, ένα κεσεδάκι γιαούρτι των 200γρ, μια φέτα τυρί του τοστ)  καταναλώνει  το παιδί σας (μέσος όρος); - v",
                           "Πόσες μερίδες δημητριακών καταναλώνει  το παιδί σας (μέσος όρος); - v",
                           "Από τις παραπάνω  μερίδες δημητριακών που καταναλώνει συνήθως το παιδί σας ημερησίως πόσες περιέχουν δημητριακά ολικής άλεσης όπως τοστ ολικής άλεσης, ψωμί ολικής άλεσης  ή πολύσπορο, παξιμάδια ολικής άλεσης, ζυμαρικά ολικής άλεσης, ποπ κορν, δημητριακά πρωινού με περιεκτικότητα σε φυτικές ίνες > 3γρ / 100γρ προϊόντος: - v",
                           "Πόσο συχνά  καταναλώνει το παιδί σας λευκό κρέας (κοτόπουλο, γαλοπούλα, πάπια, κουνέλι, κυνήγι); - v",
                           "Πόσο συχνά  καταναλώνει το παιδί σας κόκκινο κρέας (μοσχάρι, χοιρινό, κατσίκι, αρνί); - v",
                           "Πόσες φορές την εβδομάδα κατανάλωσε το παιδί σας επεξεργασμένο κρέας (π.χ. μια φέτα ζαμπόν, μια φέτα γαλοπούλα, μια φέτα σαλάμι, μια φέτα παριζάκι , ένα μεσαίο λουκάνικο) - v",
                           "Εάν το παιδί σας καταναλώνει επεξεργασμένο κρέας προσδιορίστε το είδος: - v",
                           "Το παιδί μου έχει μεγάλη όρεξη - v",
                           "Πόσο συχνά τρώει το παιδί σας λαδερό φαγητό (π.χ. φασολάκια πράσινα, αρακά, μπριάμ); - v",
                           "Το παιδί μου ΔΕΝ χορταίνει εύκολα - v",
                           "Το παιδί μου ΔΕΝ αφήνει φαγητό στο πιάτο στο τέλος του γεύματος - v",
                           "Πόσο συχνά τρώει το παιδί σας έτοιμο φαγητό ταχυφαγείου (π.χ. πίτα με γύρο, πίτσα); - v",
                           "Το παιδί μου απορρίπτει νέα φαγητά στην αρχή - v",
                           "Πόσο συχνά καταναλώνει το παιδί σας αυγά (σε οποιαδήποτε μορφή, βραστά, τηγανητά, ομελέτα, σε συνταγές); - v",
                           "Το παιδί μου μπορεί να φάει ένα γεύμα αν έχει φάει σνακ λίγο πριν - v",
                           "Πόσο συχνά πίνει το παιδί σας  αναψυκτικά τύπου coca cola ή fanta (330ml)  ή χυμούς εμπορίου π.χ. αμίτα  (1 ποτήρι των 250 ml); - v",
                           "Το παιδί μου ΔΕΝ ενδιαφέρεται να δοκιμάσει νέα φαγητά - v",
                           "Πόσο συχνά τρώει το παιδί σας τυποποιημένα προϊόντα με πολλές θερμίδες (δηλαδή >250kcal ανά 100γρ. προϊόντος) και υψηλά σε απλά σάκχαρα (δηλαδή με σάκχαρα > 22,5γρ. ανά 100γρ προϊόντος) ή υψηλά σε αλάτι (δηλαδή > 1,5γρ. αλατιού ή 0,6γρ νατρίου ανά 100 γρ προϊόντος) (π.χ. 1 σακούλι πατατάκια 70γρ, 1 κρουασάν 100γρ, 1 ντόνατς 100γρ, 1 σοκολάτα 100γρ, ≥ 3 μπισκότα τύπου γεμιστά) - v",
                           "Το παιδί μου δύσκολα ικανοποιείται με τα γεύματα - v",
                           "ΟΙΚΟΓΕΝΕΙΑΚΑ ΓΕΥΜΑΤΑ - v",
                           "ΡΥΘΜΟΣ ΣΙΤΙΣΗΣ - v",
                           "ΑΡΙΘΜΟΣ ΚΥΡΙΩΝ ΓΕΥΜΑΤΩΝ - v",
                           ]]
#---------------------------------------------------------------------------------------------------------------------------------

In [ ]:
food_habits_plt

,pid,ΜΕΣΟΔΙΑΣΤΗΜΑΤΑ ΜΕΤΑΞΥ ΓΕΥΜΑΤΩΝ: - v,ΑΡΙΘΜΟΣ ΜΙΚΡΟΓΕΥΜΑΤΩΝ / ΣΝΑΚ - v,Πόσο συχνά τρώει το παιδί σας ψάρια ή θαλασσινά; - v,ΠΡΟΣΩΠΑ ΦΡΟΝΤΙΔΑΣ ΣΙΤΙΣΗΣ ΠΑΙΔΙΟΥ: - v,Εάν έφαγε ψάρι ή θαλασσινό το παιδί σας τον προηγούμενο μήνα προσδιορίστε το είδος: - v,Πόσο συχνά χρησιμοποιείτε ελαιόλαδο στο μαγείρεμα; - v,Το παιδί μου ζητάει συνεχώς φαγητό - v,Πόσο συχνά τρώει το παιδί σας όσπρια; - v,Πόσο συχνά χρησιμοποιείτε βούτυρο; - v,...,Το παιδί μου απορρίπτει νέα φαγητά στην αρχή - v,"Πόσο συχνά καταναλώνει το παιδί σας αυγά (σε οποιαδήποτε μορφή, βραστά, τηγανητά, ομελέτα, σε συνταγές); - v",Το παιδί μου μπορεί να φάει ένα γεύμα αν έχει φάει σνακ λίγο πριν - v,Πόσο συχνά πίνει το παιδί σας αναψυκτικά τύπου coca cola ή fanta (330ml) ή χυμούς εμπορίου π.χ. αμίτα (1 ποτήρι των 250 ml); - v,Το παιδί μου ΔΕΝ ενδιαφέρεται να δοκιμάσει νέα φαγητά - v,"Πόσο συχνά τρώει το παιδί σας τυποποιημένα προϊόντα με πολλές θερμίδες (δηλαδή >250kcal ανά 100γρ. προϊόντος) και υψηλά σε απλά σάκχαρα (δηλαδή με σάκχαρα > 22,5γρ. ανά 100γρ προϊόντος) ή υψηλά σε αλάτι (δηλαδή > 1,5γρ. αλατιού ή 0,6γρ νατρίου ανά 100 γρ προϊόντος) (π.χ. 1 σακούλι πατατάκια 70γρ, 1 κρουασάν 100γρ, 1 ντόνατς 100γρ, 1 σοκολάτα 100γρ, ≥ 3 μπισκότα τύπου γεμιστά) - v",Το παιδί μου δύσκολα ικανοποιείται με τα γεύματα - v,ΟΙΚΟΓΕΝΕΙΑΚΑ ΓΕΥΜΑΤΑ - v,ΡΥΘΜΟΣ ΣΙΤΙΣΗΣ - v,ΑΡΙΘΜΟΣ ΚΥΡΙΩΝ ΓΕΥΜΑΤΩΝ - v
0,61ea9650af99b3594fb760ca,0,2,2,"0,1",3,5,2,2,0,...,4,3,2,2,4,1,3,7,1,3
1,61eab033af99b3594fb760cc,1,2,1,"0,2",9,6,4,2,2,...,2,2,3,1,2,4,2,7,1,3
2,61eab41baf99b3594fb760ce,0,2,2,"0,3","0,Γαύρος,3,4,6,7,8",6,4,2,0,...,2,3,4,0,2,3,2,7,0,3
3,61c192f96a9a5d196c5cc734,0,2,1,"0,3","3,10",5,4,2,0,...,4,4,4,0,4,2,4,2,0,3
4,61c193b26a9a5d196c5cc736,1,3,2,"0,2","Γαύρος,2,4,6",6,4,2,1,...,3,2,2,0,4,4,2,5,0,3
5,61c194936a9a5d196c5cc738,0,2,1,0,"6,7,10",6,3,2,0,...,4,3,3,1,3,2,4,2,1,2
6,61c195f56a9a5d196c5cc73a,1,4,2,"0,1",4,6,5,2,0,...,2,4,5,4,3,1,2,7,0,3
7,61d6cc7a5828a5043783a1f1,0,2,0,"0,1,2",NaN,6,3,2,0,...,3,2,4,1,3,2,3,5,1,3
8,61d6cd965828a5043783a1f3,0,2,0,0,NaN,5,3,2,0,...,2,2,3,3,4,3,2,2,0,3
9,61d6ce215828a5043783a1f5,1,3,2,"0,2","0,3,4,6,7,8",5,4,2,0,...,3,3,3,0,2,4,2,5,1,2


In [ ]:
dsfoodHabitsplt = pd.DataFrame()

In [ ]:
characteristic = 'Μεσοδιαστήματα μεταξύ γευμάτων'
number = generate_descriptive_statistics(food_habits_plt["ΜΕΣΟΔΙΑΣΤΗΜΑΤΑ ΜΕΤΑΞΥ ΓΕΥΜΑΤΩΝ: - v"],['ΣΤΑΘΕΡΑ', 'ΑΚΑΝΟΝΙΣΤΑ'],'categorical2',[0,1])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Αριθμός μιρκογευμάτων/σνακ'
number = generate_descriptive_statistics(food_habits_plt["ΑΡΙΘΜΟΣ ΜΙΚΡΟΓΕΥΜΑΤΩΝ / ΣΝΑΚ - v"],['2', '3', '4', '5'],'categorical2',[2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά τρώει το παιδί ψάρια ή θαλασσινά;'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά τρώει το παιδί σας ψάρια ή θαλασσινά; - v"],['Ποτέ/σπάνια', '1-3 φ/μήνα', '1-2 φ/εβδομ'],'categorical2',[0,1,2])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά χρησιμοποιείτε ελαιόλαδο στο μαγείρεμα;'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά χρησιμοποιείτε ελαιόλαδο στο μαγείρεμα; - v"],['1-2 φ/ημέρα', '3-4 φ/ημέρα'],'categorical2',[5,6])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Το παιδί μου ζητάει συνεχώς φαγητό'
number = generate_descriptive_statistics(food_habits_plt["Το παιδί μου ζητάει συνεχώς φαγητό - v"],['ΣΠΑΝΙΑ', 'ΜΕΡΙΚΕΣ ΦΟΡΕΣ','ΣΥΧΝΑ','ΠΑΝΤΑ'],'categorical2',[2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά τρώει το παιδί σας όσπρια;'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά τρώει το παιδί σας όσπρια; - v"],['1-3 φ/μήνα', '1-2 φ/εβδομ'],'categorical2',[1,2])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά χρησιμοποιείτε βούτυρο;'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά χρησιμοποιείτε βούτυρο; - v"],['Ποτέ/σπάνια', '1-3 φ/μήνα', '1-2 φ/εβδομ'],'categorical2',[0,1,2])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Ακόμη κι αν έχει χορτάσει, το παιδί μου βρίσκει χώρο για το αγαπημένο του φαγητό'
number = generate_descriptive_statistics(food_habits_plt["Ακόμη κι αν έχει χορτάσει, το παιδί μου βρίσκει χώρο για το αγαπημένο του φαγητό - v"],['ΣΠΑΝΙΑ', 'ΜΕΡΙΚΕΣ ΦΟΡΕΣ', 'ΣΥΧΝΑ','ΠΑΝΤΑ'],'categorical2',[2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά χρησιμοποιείτε άλλα φυτικά έλαια στο μαγείρεμα;'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά χρησιμοποιείτε άλλα φυτικά έλαια στο μαγείρεμα; - v"],['Ποτέ/σπάνια', '1-3 φ/μήνα', '1-2 φ/εβδομ','3-4 φ/εβδομ'],'categorical2',[0,1,2,3])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Εάν είχε την επιλογή, το παιδί μου θα έτρωγε την περισσότερη ώρα της ημέρας'
number = generate_descriptive_statistics(food_habits_plt["Εάν είχε την επιλογή, το παιδί μου θα έτρωγε την περισσότερη ώρα της ημέρας - v"],['ΣΠΑΝΙΑ', 'ΜΕΡΙΚΕΣ ΦΟΡΕΣ', 'ΣΥΧΝΑ','ΠΑΝΤΑ'],'categorical2',[2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά τρώει το παιδί σας ξηρούς καρπούς;'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά τρώει το παιδί σας ξηρούς καρπούς (μια μερίδα αντιστοιχεί σε μια χούφτα ξηρών καρπών (π.χ. 18 αμύγδαλα, 6 ολόκληρα καρύδια, 3 κουταλιές της σούπας ηλιόσπορους)  ή προϊόντα τους  όπως ταχίνι, φυστικοβούτυρο κ.ά. (μια μερίδα αντιστοιχεί σε 1½  κουταλιά της σούπας των 15ml ή 25γρ. π.χ. ταχίνι) - v"],['Ποτέ/σπάνια', '1-3 φ/μήνα', '1-2 φ/εβδομ','3-4 φ/εβδομ'],'categorical2',[0,1,2,3])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά χρησιμοποιείτε μαργαρίνη;'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά χρησιμοποιείτε μαργαρίνη; - v"],['Ποτέ/σπάνια','1-3 φ/μήνα','1-2 φ/εβδομ','5-7 φ/εβδ'],'categorical2',[0,1,2,4])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες μερίδες φρούτων καταναλώνει το παιδί σας (μέσος όρος);'
number = generate_descriptive_statistics(food_habits_plt["Πόσες μερίδες φρούτων καταναλώνει το παιδί σας (μέσος όρος); - v"],['1-2 φ/εβδομ','3-4 φ/εβδομ','1-2 φ/ημέρα','3-4 φ/ημέρα'],'categorical2',[2,3,5,6])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες μερίδες λαχανικών (ωμών ή βραστών ή στον ατμό ή ψητών) καταναλώνει το παιδί σας (μέσος όρος);'
number = generate_descriptive_statistics(food_habits_plt["Πόσες μερίδες λαχανικών (ωμών ή βραστών ή στον ατμό ή ψητών) καταναλώνει το παιδί σας (μέσος όρος); - v"],['1-2 φ/εβδομ','3-4 φ/εβδομ','1-2 φ/ημέρα','3-4 φ/ημέρα'],'categorical2',[2,3,5,6])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες μερίδες γαλακτοκομικών καταναλώνει  το παιδί σας (μέσος όρος);'
number = generate_descriptive_statistics(food_habits_plt["Πόσες μερίδες γαλακτοκομικών (ένα ποτήρι γάλα των 250ml, ένα κεσεδάκι γιαούρτι των 200γρ, μια φέτα τυρί του τοστ)  καταναλώνει  το παιδί σας (μέσος όρος); - v"],['3-4 φ/εβδομ','1-2 φ/ημέρα','3-4 φ/ημέρα'],'categorical2',[3,5,6])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)                   

characteristic = 'Πόσες μερίδες δημητριακών καταναλώνει  το παιδί σας (μέσος όρος);'
number = generate_descriptive_statistics(food_habits_plt["Πόσες μερίδες δημητριακών καταναλώνει  το παιδί σας (μέσος όρος); - v"],['7-8 μερίδες/μέρα','9+ μερίδες/μέρα'],'categorical2',[3,4])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)       

characteristic = 'Από τις παραπάνω  μερίδες δημητριακών που καταναλώνει συνήθως το παιδί σας ημερησίως πόσες περιέχουν δημητριακά ολικής άλεσης;'
number = generate_descriptive_statistics(food_habits_plt["Από τις παραπάνω  μερίδες δημητριακών που καταναλώνει συνήθως το παιδί σας ημερησίως πόσες περιέχουν δημητριακά ολικής άλεσης όπως τοστ ολικής άλεσης, ψωμί ολικής άλεσης  ή πολύσπορο, παξιμάδια ολικής άλεσης, ζυμαρικά ολικής άλεσης, ποπ κορν, δημητριακά πρωινού με περιεκτικότητα σε φυτικές ίνες > 3γρ / 100γρ προϊόντος: - v"],['1-2 μερίδες/μέρα','3-4 μερίδες/μέρα','5-6 μερίδες/μέρα','Καμία'],'categorical2',[0,1,2,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)       

characteristic = 'Πόσο συχνά  καταναλώνει το παιδί σας λευκό κρέας (κοτόπουλο, γαλοπούλα, πάπια, κουνέλι, κυνήγι);'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά  καταναλώνει το παιδί σας λευκό κρέας (κοτόπουλο, γαλοπούλα, πάπια, κουνέλι, κυνήγι); - v"],['1-2 φ/εβδομ', '3-4 φ/εβδομ'],'categorical2',[2,3])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά  καταναλώνει το παιδί σας κόκκινο κρέας (μοσχάρι, χοιρινό, κατσίκι, αρνί);'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά  καταναλώνει το παιδί σας κόκκινο κρέας (μοσχάρι, χοιρινό, κατσίκι, αρνί); - v"],['1-2 φ/εβδομ', '3-4 φ/εβδομ'],'categorical2',[2,3])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες φορές την εβδομάδα κατανάλωσε το παιδί σας επεξεργασμένο κρέας;'
number = generate_descriptive_statistics(food_habits_plt["Πόσες φορές την εβδομάδα κατανάλωσε το παιδί σας επεξεργασμένο κρέας (π.χ. μια φέτα ζαμπόν, μια φέτα γαλοπούλα, μια φέτα σαλάμι, μια φέτα παριζάκι , ένα μεσαίο λουκάνικο) - v"],['1-3 φ/μήνα', '1-2 φ/εβδομ','3-4 φ/εβδομ','5-7 φ/εβδ'],'categorical2',[1,2,3,4])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Το παιδί μου έχει μεγάλη όρεξη'
number = generate_descriptive_statistics(food_habits_plt["Το παιδί μου έχει μεγάλη όρεξη - v"],['ΣΠΑΝΙΑ', 'ΜΕΡΙΚΕΣ ΦΟΡΕΣ','ΣΥΧΝΑ','ΠΑΝΤΑ'],'categorical2',[2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά τρώει το παιδί σας λαδερό φαγητό;'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά τρώει το παιδί σας λαδερό φαγητό (π.χ. φασολάκια πράσινα, αρακά, μπριάμ); - v"],['Ποτέ/σπάνια', '1-3 φ/μήνα','1-2 φ/εβδομ'],'categorical2',[0,1,2])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Το παιδί μου ΔΕΝ χορταίνει εύκολα'
number = generate_descriptive_statistics(food_habits_plt["Το παιδί μου ΔΕΝ χορταίνει εύκολα - v"],['ΣΠΑΝΙΑ', 'ΜΕΡΙΚΕΣ ΦΟΡΕΣ','ΣΥΧΝΑ','ΠΑΝΤΑ'],'categorical2',[2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Το παιδί μου ΔΕΝ αφήνει φαγητό στο πιάτο στο τέλος του γεύματος'
number = generate_descriptive_statistics(food_habits_plt["Το παιδί μου ΔΕΝ αφήνει φαγητό στο πιάτο στο τέλος του γεύματος - v"],['ΣΠΑΝΙΑ', 'ΜΕΡΙΚΕΣ ΦΟΡΕΣ','ΣΥΧΝΑ','ΠΑΝΤΑ'],'categorical2',[2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά τρώει το παιδί σας έτοιμο φαγητό ταχυφαγείου (π.χ. πίτα με γύρο, πίτσα);'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά τρώει το παιδί σας έτοιμο φαγητό ταχυφαγείου (π.χ. πίτα με γύρο, πίτσα); - v"],['1-3 φ/μήνα', '1-2 φ/εβδομ','3-4 φ/εβδομ'],'categorical2',[1,2,3])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Το παιδί μου απορρίπτει νέα φαγητά στην αρχή'
number = generate_descriptive_statistics(food_habits_plt["Το παιδί μου απορρίπτει νέα φαγητά στην αρχή - v"],['ΣΠΑΝΙΑ', 'ΜΕΡΙΚΕΣ ΦΟΡΕΣ','ΣΥΧΝΑ','ΠΑΝΤΑ'],'categorical2',[2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά καταναλώνει το παιδί σας αυγά (σε οποιαδήποτε μορφή, βραστά, τηγανητά, ομελέτα, σε συνταγές);'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά καταναλώνει το παιδί σας αυγά (σε οποιαδήποτε μορφή, βραστά, τηγανητά, ομελέτα, σε συνταγές); - v"],['1-2 φ/εβδομ','3-4 φ/εβδομ','5-7 φ/εβδ'],'categorical2',[2,3,4])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Το παιδί μου μπορεί να φάει ένα γεύμα αν έχει φάει σνακ λίγο πριν'
number = generate_descriptive_statistics(food_habits_plt["Το παιδί μου μπορεί να φάει ένα γεύμα αν έχει φάει σνακ λίγο πριν - v"],['ΣΠΑΝΙΑ', 'ΜΕΡΙΚΕΣ ΦΟΡΕΣ','ΣΥΧΝΑ','ΠΑΝΤΑ'],'categorical2',[2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά πίνει το παιδί αναψυκτικά ή χυμούς εμπορίου;'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά πίνει το παιδί σας  αναψυκτικά τύπου coca cola ή fanta (330ml)  ή χυμούς εμπορίου π.χ. αμίτα  (1 ποτήρι των 250 ml); - v"],['Ποτέ/σπάνια', '1-3 φ/μήνα', '1-2 φ/εβδομ','3-4 φ/εβδομ','5-7 φ/εβδ','1-2 φ/ημέρα'],'categorical2',[0,1,2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Το παιδί μου ΔΕΝ ενδιαφέρεται να δοκιμάσει νέα φαγητά'
number = generate_descriptive_statistics(food_habits_plt["Το παιδί μου ΔΕΝ ενδιαφέρεται να δοκιμάσει νέα φαγητά - v"],['ΣΠΑΝΙΑ', 'ΜΕΡΙΚΕΣ ΦΟΡΕΣ', 'ΣΥΧΝΑ'],'categorical2',[2,3,4])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)       

characteristic = 'Πόσο συχνά τρώει το παιδί σας τυποποιημένα προϊόντα με πολλές θερμίδες;'
number = generate_descriptive_statistics(food_habits_plt["Πόσο συχνά τρώει το παιδί σας τυποποιημένα προϊόντα με πολλές θερμίδες (δηλαδή >250kcal ανά 100γρ. προϊόντος) και υψηλά σε απλά σάκχαρα (δηλαδή με σάκχαρα > 22,5γρ. ανά 100γρ προϊόντος) ή υψηλά σε αλάτι (δηλαδή > 1,5γρ. αλατιού ή 0,6γρ νατρίου ανά 100 γρ προϊόντος) (π.χ. 1 σακούλι πατατάκια 70γρ, 1 κρουασάν 100γρ, 1 ντόνατς 100γρ, 1 σοκολάτα 100γρ, ≥ 3 μπισκότα τύπου γεμιστά) - v"],['1-3 φ/μήνα', '1-2 φ/εβδομ','3-4 φ/εβδομ','5-7 φ/εβδ','1-2 φ/ημέρα'],'categorical2',[1,2,3,4,5])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)       

characteristic = 'Το παιδί μου δύσκολα ικανοποιείται με τα γεύματα'
number = generate_descriptive_statistics(food_habits_plt["Το παιδί μου δύσκολα ικανοποιείται με τα γεύματα - v"],['ΣΠΑΝΙΑ', 'ΜΕΡΙΚΕΣ ΦΟΡΕΣ', 'ΣΥΧΝΑ'],'categorical2',[2,3,4])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)    
                        
characteristic = 'Αριθμός οικογενειακών γευμάτων'
number = generate_descriptive_statistics(food_habits_plt["ΟΙΚΟΓΕΝΕΙΑΚΑ ΓΕΥΜΑΤΑ - v"],['1', '2', '5','7'],'categorical2',[1,2,5,7])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)    

characteristic = 'Ρυθμός σίτισης'
number = generate_descriptive_statistics(food_habits_plt["ΡΥΘΜΟΣ ΣΙΤΙΣΗΣ - v"],['ΓΡΗΓΟΡΑ', 'ΑΡΓΑ.'],'categorical2',[0,1])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)   

characteristic = 'Αριθμός κύριων γευμάτων'
number = generate_descriptive_statistics(food_habits_plt["ΑΡΙΘΜΟΣ ΚΥΡΙΩΝ ΓΕΥΜΑΤΩΝ - v"],['2','3'],'categorical2',[2,3])
dsfoodHabitsplt = dsfoodHabitsplt.append({C:characteristic,N:number},ignore_index=True)   


In [ ]:
dsfoodHabitsplt

,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Μεσοδιαστήματα μεταξύ γευμάτων,"ΣΤΑΘΕΡΑ: 8(53.33%), ΑΚΑΝΟΝΙΣΤΑ: 7(46.67%)"
1,Αριθμός μιρκογευμάτων/σνακ,"2: 10(66.67%), 3: 3(20.0%), 4: 1(6.67%), 5: 1(..."
2,Πόσο συχνά τρώει το παιδί ψάρια ή θαλασσινά;,"Ποτέ/σπάνια: 2(13.33%), 1-3 φ/μήνα: 6(40.0%), ..."
3,Πόσο συχνά χρησιμοποιείτε ελαιόλαδο στο μαγείρ...,"1-2 φ/ημέρα: 7(46.67%), 3-4 φ/ημέρα: 8(53.33%)"
4,Το παιδί μου ζητάει συνεχώς φαγητό,"ΣΠΑΝΙΑ: 2(13.33%), ΜΕΡΙΚΕΣ ΦΟΡΕΣ: 3(20.0%), ΣΥ..."
5,Πόσο συχνά τρώει το παιδί σας όσπρια;,"1-3 φ/μήνα: 1(6.67%), 1-2 φ/εβδομ: 14(93.33%)"
6,Πόσο συχνά χρησιμοποιείτε βούτυρο;,"Ποτέ/σπάνια: 12(80.0%), 1-3 φ/μήνα: 2(13.33%),..."
7,"Ακόμη κι αν έχει χορτάσει, το παιδί μου βρίσκε...","ΣΠΑΝΙΑ: 2(13.33%), ΜΕΡΙΚΕΣ ΦΟΡΕΣ: 6(40.0%), ΣΥ..."
8,Πόσο συχνά χρησιμοποιείτε άλλα φυτικά έλαια στ...,"Ποτέ/σπάνια: 10(66.67%), 1-3 φ/μήνα: 3(20.0%),..."
9,"Εάν είχε την επιλογή, το παιδί μου θα έτρωγε τ...","ΣΠΑΝΙΑ: 4(26.67%), ΜΕΡΙΚΕΣ ΦΟΡΕΣ: 5(33.33%), Σ..."


In [ ]:
#----------------------------------------Food habits pre-pilot---------------------------------------------
#food_habits1 = retrieve_data('questionnaireData.xlsx','foodhabits')
food_habits_pre = retrieve_data('questionnaireData_ctrlpre.xlsx','foodhabits')
#food_habits = pd.concat([food_habits1, food_habits2], axis=0)
#food_habits.drop(['Unnamed: 0'],axis=1,inplace=True)
#food_habits.reset_index(drop=True, inplace=True)

food_habits_pre = food_habits_pre[["pid",
                           "Από τις παραπάνω  μερίδες δημητριακών που κατανάλωσε το παιδί σας ημερησίως πόσες περιείχαν δημητριακά ολικής άλεσης όπως τοστ ολικής άλεσης, ψωμί ολικής άλεσης  ή πολύσπορο, παξιμάδια ολικής άλεσης, ζυμαρικά ολικής άλεσης, ποπ κορν, δημητριακά πρωινού με περιεκτικότητα σε φυτικές ίνες > 3γρ / 100γρ προϊόντος: - v",
                           "Πόσα αυγά έφαγε το παιδί σας την εβδομάδα (σε οποιαδήποτε μορφή, βραστά, τηγανητά, ομελέτα, σε συνταγές); - v",
                           "Πόσες μερίδες γαλακτοκομικών (ένα ποτήρι γάλα των 250ml, ένα κεσεδάκι γιαούρτι των 200γρ, μια φέτα τυρί του τοστ)  κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v",
                           "Πόσες μερίδες δημητριακών κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v",
                           "Πόσες μερίδες λαχανικών (ωμών ή βραστών ή στον ατμό ή ψητών) κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v",
                           "Πόσες μερίδες φρούτων κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v",
                           "Πόσες φορές την εβδομάδα έφαγε το παιδί σας κόκκινο κρέας (μοσχάρι, χοιρινό, κατσίκι, αρνί); - v",
                           "Πόσες φορές την εβδομάδα έφαγε το παιδί σας λευκό κρέας (κοτόπουλο, γαλοπούλα, πάπια, κουνέλι, κυνήγι); - v",
                           "Πόσες φορές την εβδομάδα έφαγε το παιδί σας τυποποιημένα προϊόντα με πολλές θερμίδες (δηλαδή >250kcal ανά 100γρ. προϊόντος) και υψηλά σε απλά σάκχαρα (δηλαδή με σάκχαρα > 22,5γρ. ανά 100γρ προϊόντος) ή υψηλά σε αλάτι (δηλαδή > 1,5γρ. αλατιού ή 0,6γρ νατρίου ανά 100 γρ προϊόντος) (π.χ. 1 σακούλι πατατάκια 100γρ, 1 κρουασάν 100γρ, 1 ντόνατς 100γρ, 1 σοκολάτα 100γρ, ≥ 3 μπισκότα τύπου γεμιστά) - v",
                           "Πόσες φορές την εβδομάδα ήπιε το παιδί σας  αναψυκτικά τύπου coca cola ή fanta ή χυμούς εμπορίου π.χ. αμίτα  (1 ποτήρι των 250 ml); - v",
                           "Πόσες φορές την εβδομάδα ασκήθηκε (οργανωμένο σπορ) το παιδί σας; - v",
                           "Πόσες φορές την εβδομάδα κατανάλωσε το παιδί σας επεξεργασμένο κρέας (π.χ. μια φέτα ζαμπόν, μια φέτα γαλοπούλα, μια φέτα σαλάμι, μια φέτα παριζάκι , ένα μεσαίο λουκάνικο) - v",
                           "Πόσο συχνά έφαγε το παιδί σας λαδερό φαγητό (π.χ. φασολάκια πράσινα, αρακά, μπριάμ) την εβδομάδα που μας πέρασε; - v",
                           "Πόσο συχνά έφαγε το παιδί σας την τελευταία βδομάδα έτοιμο φαγητό ταχυφαγείου (π.χ. πίτα με γύρο, πίτσα); - v",
                           "Πόσο συχνά έφαγε το παιδί σας ψάρια ή θαλασσινά την εβδομάδα που μας πέρασε; - v",
                           "Πόσο συχνά έφαγε το παιδί σας όσπρια την εβδομάδα που μας πέρασε; - v",
                           "Πόσο συχνά χρησιμοποιήσατε άλλα φυτικά έλαια στο μαγείρεμα; - v",
                           "Πόσο συχνά χρησιμοποιήσατε βούτυρο τη βδομάδα; - v",
                           "Πόσο συχνά χρησιμοποιήσατε ελαιόλαδο στο μαγείρεμα; - v",
                           "Πόσο συχνά χρησιμοποιήσατε μαργαρίνη τη βδομάδα; - v",
                           "Πόσο συχνά έφαγε το παιδί σας ξηρούς καρπούς (μια μερίδα αντιστοιχεί σε μια χούφτα ξηρών καρπών (π.χ. 18 αμύγδαλα, 6 ολόκληρα καρύδια, 3 κουταλιές της σούπας ηλιόσπορους)  ή προϊόντα τους  όπως ταχίνι, φυστικοβούτυρο κ.ά. (μια μερίδα αντιστοιχεί σε 1½  κουταλιά της σούπας των 15ml ή 25γρ. π.χ. ταχίνι) - v"]]
#---------------------------------------------------------------------------------------------------------------------------------

In [ ]:
food_habits_pre

,pid,"Από τις παραπάνω μερίδες δημητριακών που κατανάλωσε το παιδί σας ημερησίως πόσες περιείχαν δημητριακά ολικής άλεσης όπως τοστ ολικής άλεσης, ψωμί ολικής άλεσης ή πολύσπορο, παξιμάδια ολικής άλεσης, ζυμαρικά ολικής άλεσης, ποπ κορν, δημητριακά πρωινού με περιεκτικότητα σε φυτικές ίνες > 3γρ / 100γρ προϊόντος: - v",Εάν έφαγε ψάρι ή θαλασσινό το παιδί σας την προηγούμενη βδομάδα προσδιορίστε το είδος: - v,Εάν το παιδί σας καταναλώνει επεξεργασμένο κρέας προσδιορίστε το είδος: - v,"Πόσα αυγά έφαγε το παιδί σας την εβδομάδα (σε οποιαδήποτε μορφή, βραστά, τηγανητά, ομελέτα, σε συνταγές); - v","Πόσα λεπτά την ημέρα περίπου ήταν το παιδί σας σωματικά δραστήριο (περπάτημα, ποδήλατο, πατίνι, παιχνίδια με φίλους που απαιτούν σωματική δραστηριότητα, κηπουρική, μαγειρική, δουλειές σπιτιού) - v","Πόσες μερίδες γαλακτοκομικών (ένα ποτήρι γάλα των 250ml, ένα κεσεδάκι γιαούρτι των 200γρ, μια φέτα τυρί του τοστ) κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v",Πόσες μερίδες δημητριακών κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v,Πόσες μερίδες λαχανικών (ωμών ή βραστών ή στον ατμό ή ψητών) κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v,Πόσες μερίδες φρούτων κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v,...,"Πόσο συχνά έφαγε το παιδί σας λαδερό φαγητό (π.χ. φασολάκια πράσινα, αρακά, μπριάμ) την εβδομάδα που μας πέρασε; - v","Πόσο συχνά έφαγε το παιδί σας την τελευταία βδομάδα έτοιμο φαγητό ταχυφαγείου (π.χ. πίτα με γύρο, πίτσα); - v",Πόσο συχνά έφαγε το παιδί σας ψάρια ή θαλασσινά την εβδομάδα που μας πέρασε; - v,Πόσο συχνά έφαγε το παιδί σας όσπρια την εβδομάδα που μας πέρασε; - v,Πόσο συχνά χρησιμοποιήσατε άλλα φυτικά έλαια στο μαγείρεμα; - v,Πόσο συχνά χρησιμοποιήσατε βούτυρο τη βδομάδα; - v,Πόσο συχνά χρησιμοποιήσατε ελαιόλαδο στο μαγείρεμα; - v,Πόσο συχνά χρησιμοποιήσατε μαργαρίνη τη βδομάδα; - v,Τι είδους άσκηση έκανε το παιδί σας την προηγούμενη βδομάδα; - v,"Πόσο συχνά έφαγε το παιδί σας ξηρούς καρπούς (μια μερίδα αντιστοιχεί σε μια χούφτα ξηρών καρπών (π.χ. 18 αμύγδαλα, 6 ολόκληρα καρύδια, 3 κουταλιές της σούπας ηλιόσπορους) ή προϊόντα τους όπως ταχίνι, φυστικοβούτυρο κ.ά. (μια μερίδα αντιστοιχεί σε 1½ κουταλιά της σούπας των 15ml ή 25γρ. π.χ. ταχίνι) - v"
0,6058391a4cbd8a20b68b237c,2.0,5,1,2.0,3,1.0,4.0,1.0,1.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,7,NaN
1,60584c5f4cbd8a20b68b237e,0.0,7,1,1.0,1,2.0,2.0,1.0,1.0,...,0.0,1.0,1.0,1.0,3.0,2.0,1.0,0.0,3,NaN
2,605987294cbd8a20b68b2380,0.0,3,NaN,0.0,0,1.0,0.0,0.0,0.0,...,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,7,0.0
3,60598ea24cbd8a20b68b2382,0.0,"3,7","1,4",3.0,4,1.0,4.0,1.0,0.0,...,0.0,3.0,2.0,1.0,0.0,0.0,1.0,3.0,7,4.0
4,60616dc34cbd8a20b68b2384,0.0,"4,6,8",1,1.0,3,1.0,0.0,0.0,1.0,...,0.0,1.0,2.0,1.0,0.0,0.0,1.0,3.0,3,2.0
5,6061786a4cbd8a20b68b2386,1.0,3,2,1.0,0,1.0,0.0,0.0,1.0,...,1.0,3.0,1.0,2.0,0.0,3.0,1.0,0.0,7,1.0
6,6062c0f4fc6f0f5fdc58111f,0.0,"Γαύρος,3",1,1.0,2,1.0,3.0,1.0,1.0,...,1.0,3.0,1.0,0.0,0.0,3.0,1.0,3.0,0,0.0
7,6062c75bfc6f0f5fdc581121,2.0,4,1,1.0,2,2.0,2.0,1.0,1.0,...,2.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,7,1.0
8,606bfb78dbba1465ad26c107,1.0,"Γαύρος,7,8","0,1",1.0,2,1.0,2.0,2.0,1.0,...,1.0,1.0,2.0,2.0,3.0,0.0,1.0,3.0,7,4.0
9,606c0ed6dbba1465ad26c109,0.0,"0,Γαύρος,3",1,1.0,2,2.0,1.0,1.0,1.0,...,1.0,1.0,2.0,2.0,0.0,0.0,1.0,3.0,7,0.0


In [ ]:
food_habits_pre=food_habits_pre.dropna()
dsfoodHabitspreplt = pd.DataFrame()

In [ ]:
characteristic = 'Πόσες μερίδες δημητριακών κατανάλωσε το παιδί σας την ημέρα (μέσος όρος);'
number = generate_descriptive_statistics(food_habits_pre["Πόσες μερίδες δημητριακών κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v"],['1-2 μερίδες/μέρα','3-4 μερίδες/μέρα','5-6 μερίδες/μέρα','7-8 μερίδες/μέρα','9+ μερίδες/μέρα'],'categorical2',[0,1,2,3,4])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Από τις παραπάνω  μερίδες δημητριακών που κατανάλωσε το παιδί σας ημερησίως πόσες περιείχαν δημητριακά ολικής άλεσης;'
number = generate_descriptive_statistics(food_habits_pre["Από τις παραπάνω  μερίδες δημητριακών που κατανάλωσε το παιδί σας ημερησίως πόσες περιείχαν δημητριακά ολικής άλεσης όπως τοστ ολικής άλεσης, ψωμί ολικής άλεσης  ή πολύσπορο, παξιμάδια ολικής άλεσης, ζυμαρικά ολικής άλεσης, ποπ κορν, δημητριακά πρωινού με περιεκτικότητα σε φυτικές ίνες > 3γρ / 100γρ προϊόντος: - v"],['1-2 μερίδες/μέρα','3-4 μερίδες/μέρα','5-6 μερίδες/μέρα','9+ μερίδες/μέρα'],'categorical2',[0,1,2,4])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσα αυγά έφαγε το παιδί σας την εβδομάδα;'
number = generate_descriptive_statistics(food_habits_pre["Πόσα αυγά έφαγε το παιδί σας την εβδομάδα (σε οποιαδήποτε μορφή, βραστά, τηγανητά, ομελέτα, σε συνταγές); - v"],['Καθόλου', '1 – 2 αυγά / βδ.','3 – 4 αυγά / βδ.','5 – 6 αυγά / βδ.','7+ αυγά / βδ.'],'categorical2',[0,1,2,3,3])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες μερίδες γαλακτοκομικών;'
number = generate_descriptive_statistics(food_habits_pre["Πόσες μερίδες γαλακτοκομικών (ένα ποτήρι γάλα των 250ml, ένα κεσεδάκι γιαούρτι των 200γρ, μια φέτα τυρί του τοστ)  κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v"],['1-2 μερίδες/μέρα', '3-4 μερίδες/μέρα', '5+ μερίδες/μέρα'],'categorical2',[1,2,3])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες μερίδες λαχανικών κατανάλωσε το παιδί σας την ημέρα (μέσος όρος);'
number = generate_descriptive_statistics(food_habits_pre["Πόσες μερίδες λαχανικών (ωμών ή βραστών ή στον ατμό ή ψητών) κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v"],['Καθόλου', '1-2 μερίδες/μέρα', '3-4 μερίδες/μέρα'],'categorical2',[0,1,2])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες μερίδες φρούτων κατανάλωσε το παιδί σας την ημέρα (μέσος όρος);'
number = generate_descriptive_statistics(food_habits_pre["Πόσες μερίδες φρούτων κατανάλωσε το παιδί σας την ημέρα (μέσος όρος); - v"],['Καθόλου', '1-2 μερίδες/μέρα', '3-4 μερίδες/μέρα'],'categorical2',[0,1,2])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες φορές την εβδομάδα έφαγε το παιδί σας κόκκινο κρέας (μοσχάρι, χοιρινό, κατσίκι, αρνί);'
number = generate_descriptive_statistics(food_habits_pre["Πόσες φορές την εβδομάδα έφαγε το παιδί σας κόκκινο κρέας (μοσχάρι, χοιρινό, κατσίκι, αρνί); - v"],['Καθόλου','1-2 φ/εβδομ','3-4 φ/εβδομ'],'categorical2',[0,1,2])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)


characteristic = 'Πόσες φορές την εβδομάδα έφαγε το παιδί σας λευκό κρέας (κοτόπουλο, γαλοπούλα, πάπια, κουνέλι, κυνήγι);'
number = generate_descriptive_statistics(food_habits_pre["Πόσες φορές την εβδομάδα έφαγε το παιδί σας λευκό κρέας (κοτόπουλο, γαλοπούλα, πάπια, κουνέλι, κυνήγι); - v"],['1-2 φ/εβδομ','3-4 φ/εβδομ'],'categorical2',[1,2])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες φορές την εβδομάδα έφαγε το παιδί σας τυποποιημένα προϊόντα με πολλές θερμίδες;'
number = generate_descriptive_statistics(food_habits_pre["Πόσες φορές την εβδομάδα έφαγε το παιδί σας τυποποιημένα προϊόντα με πολλές θερμίδες (δηλαδή >250kcal ανά 100γρ. προϊόντος) και υψηλά σε απλά σάκχαρα (δηλαδή με σάκχαρα > 22,5γρ. ανά 100γρ προϊόντος) ή υψηλά σε αλάτι (δηλαδή > 1,5γρ. αλατιού ή 0,6γρ νατρίου ανά 100 γρ προϊόντος) (π.χ. 1 σακούλι πατατάκια 100γρ, 1 κρουασάν 100γρ, 1 ντόνατς 100γρ, 1 σοκολάτα 100γρ, ≥ 3 μπισκότα τύπου γεμιστά) - v"],['Καθόλου ή < 1 φορά /  βδ','1 – 2 φορές / βδ','3 – 4 φορές / βδ','> 1 φορά / ημέρα','5– 7 φορές / βδ.'],'categorical2',[0,1,2,3,3])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες φορές την εβδομάδα ήπιε το παιδί σας αναψυκτικά ή χυμούς εμπορίου;'
number = generate_descriptive_statistics(food_habits_pre["Πόσες φορές την εβδομάδα ήπιε το παιδί σας  αναψυκτικά τύπου coca cola ή fanta ή χυμούς εμπορίου π.χ. αμίτα  (1 ποτήρι των 250 ml); - v"],['Καθόλου ή < 1 φορά /  βδ','1 – 2 φορές / βδ','3 – 4 φορές / βδ'],'categorical2',[0,1,2])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσες φορές την εβδομάδα κατανάλωσε το παιδί σας επεξεργασμένο κρέας;'
number = generate_descriptive_statistics(food_habits_pre["Πόσες φορές την εβδομάδα κατανάλωσε το παιδί σας επεξεργασμένο κρέας (π.χ. μια φέτα ζαμπόν, μια φέτα γαλοπούλα, μια φέτα σαλάμι, μια φέτα παριζάκι , ένα μεσαίο λουκάνικο) - v"],['Καθόλου','1-2 φορές/βδ','3-4 φορές/βδ','5 φορές/βδ','6-7 φορές/βδ'],'categorical2',[0,1,2,3,4])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά έφαγε το παιδί σας λαδερό φαγητό την εβδομάδα που μας πέρασε;'
number = generate_descriptive_statistics(food_habits_pre["Πόσο συχνά έφαγε το παιδί σας λαδερό φαγητό (π.χ. φασολάκια πράσινα, αρακά, μπριάμ) την εβδομάδα που μας πέρασε; - v"],['Καθόλου','1 φορά / βδ','2 φορές/βδ'],'categorical2',[0,1,2])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)                   

characteristic = 'Πόσο συχνά έφαγε το παιδί σας την τελευταία βδομάδα έτοιμο φαγητό ταχυφαγείου;'
number = generate_descriptive_statistics(food_habits_pre["Πόσο συχνά έφαγε το παιδί σας την τελευταία βδομάδα έτοιμο φαγητό ταχυφαγείου (π.χ. πίτα με γύρο, πίτσα); - v"],['Καθόλου','1 φορά / βδ','2 φορές/βδ','3 φορές / βδ'],'categorical2',[0,1,2,3])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)       

characteristic = 'Πόσο συχνά έφαγε το παιδί σας ψάρια ή θαλασσινά την εβδομάδα που μας πέρασε;'
number = generate_descriptive_statistics(food_habits_pre["Πόσο συχνά έφαγε το παιδί σας ψάρια ή θαλασσινά την εβδομάδα που μας πέρασε; - v"],['Καθόλου','1 φορά / βδ','2 φορές/βδ'],'categorical2',[0,1,2])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)       

characteristic = 'Πόσο συχνά έφαγε το παιδί σας όσπρια την εβδομάδα που μας πέρασε;'
number = generate_descriptive_statistics(food_habits_pre["Πόσο συχνά έφαγε το παιδί σας όσπρια την εβδομάδα που μας πέρασε; - v"],['Καθόλου', '1 φορά / βδ','2 φορές/βδ','3 φορές / βδ'],'categorical2',[0,1,2,3])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά χρησιμοποιήσατε άλλα φυτικά έλαια στο μαγείρεμα;'
number = generate_descriptive_statistics(food_habits_pre["Πόσο συχνά χρησιμοποιήσατε άλλα φυτικά έλαια στο μαγείρεμα; - v"],['Καθόλου', '3-5 φορές/βδ','1-2 φορές / βδ'],'categorical2',[0,2,3])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά χρησιμοποιήσατε βούτυρο τη βδομάδα;'
number = generate_descriptive_statistics(food_habits_pre["Πόσο συχνά χρησιμοποιήσατε βούτυρο τη βδομάδα; - v"],['Καθόλου','3-5 φορές/βδ', '1-2 φορές / βδ'],'categorical2',[0,2,3])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά χρησιμοποιήσατε ελαιόλαδο στο μαγείρεμα;'
number = generate_descriptive_statistics(food_habits_pre["Πόσο συχνά χρησιμοποιήσατε ελαιόλαδο στο μαγείρεμα; - v"],['6-7 φορές / βδ'],'categorical2',[1])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά χρησιμοποιήσατε μαργαρίνη τη βδομάδα;'
number = generate_descriptive_statistics(food_habits_pre["Πόσο συχνά χρησιμοποιήσατε μαργαρίνη τη βδομάδα; - v"],['Καθόλου', '3-5 φορές/βδ','1-2 φορές / βδ'],'categorical2',[0,2,3])
dsfoodHabitspreplt =dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Πόσο συχνά έφαγε το παιδί σας ξηρούς καρπούς;'
number = generate_descriptive_statistics(food_habits_pre["Πόσο συχνά έφαγε το παιδί σας ξηρούς καρπούς (μια μερίδα αντιστοιχεί σε μια χούφτα ξηρών καρπών (π.χ. 18 αμύγδαλα, 6 ολόκληρα καρύδια, 3 κουταλιές της σούπας ηλιόσπορους)  ή προϊόντα τους  όπως ταχίνι, φυστικοβούτυρο κ.ά. (μια μερίδα αντιστοιχεί σε 1½  κουταλιά της σούπας των 15ml ή 25γρ. π.χ. ταχίνι) - v"],['Καθόλου', '1 φορά / βδ','2 φορές/βδ','4 φορές / βδ','5-7 φορές / βδ'],'categorical2',[0,1,2,4,5])
dsfoodHabitspreplt = dsfoodHabitspreplt.append({C:characteristic,N:number},ignore_index=True)



In [ ]:
dsfoodHabitspreplt

,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Πόσες μερίδες δημητριακών κατανάλωσε το παιδί ...,"1-2 μερίδες/μέρα: 3(18.75%), 3-4 μερίδες/μέρα:..."
1,Από τις παραπάνω μερίδες δημητριακών που κατα...,"1-2 μερίδες/μέρα: 11(68.75%), 3-4 μερίδες/μέρα..."
2,Πόσα αυγά έφαγε το παιδί σας την εβδομάδα;,"Καθόλου: 1(6.25%), 1 – 2 αυγά / βδ.: 10(62.5%)..."
3,Πόσες μερίδες γαλακτοκομικών;,"1-2 μερίδες/μέρα: 10(62.5%), 3-4 μερίδες/μέρα:..."
4,Πόσες μερίδες λαχανικών κατανάλωσε το παιδί σα...,"Καθόλου: 5(31.25%), 1-2 μερίδες/μέρα: 10(62.5%..."
5,Πόσες μερίδες φρούτων κατανάλωσε το παιδί σας ...,"Καθόλου: 4(25.0%), 1-2 μερίδες/μέρα: 9(56.25%)..."
6,Πόσες φορές την εβδομάδα έφαγε το παιδί σας κό...,"Καθόλου: 0(0.0%), 1-2 φ/εβδομ: 9(56.25%), 3-4 ..."
7,Πόσες φορές την εβδομάδα έφαγε το παιδί σας λε...,"1-2 φ/εβδομ: 11(68.75%), 3-4 φ/εβδομ: 5(31.25%)"
8,Πόσες φορές την εβδομάδα έφαγε το παιδί σας τυ...,"Καθόλου ή < 1 φορά / βδ: 5(31.25%), 1 – 2 φορ..."
9,Πόσες φορές την εβδομάδα ήπιε το παιδί σας ανα...,"Καθόλου ή < 1 φορά / βδ: 8(50.0%), 1 – 2 φορέ..."


In [ ]:
dsactivity_habits = pd.DataFrame()

characteristic = 'Σωματική Δραστηριότητα'
number = generate_descriptive_statistics(food_habits["Πόσα λεπτά την ημέρα περίπου ήταν το παιδί σας σωματικά δραστήριο (περπάτημα, ποδήλατο, πατίνι, παιχνίδια με φίλους που απαιτούν σωματική δραστηριότητα, κηπουρική, μαγειρική, δουλειές σπιτιού) - v"],['Καθόλου', '< 30 λεπτά','30 – 60 λεπτά','60 – 120 λεπτά', '> 120 λεπτά'],'categorical2',[0,1,2,3,4])
dsactivity_habits = dsactivity_habits.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Οργανωμένη Άσκηση'
number = generate_descriptive_statistics(food_habits["Πόσες φορές την εβδομάδα ασκήθηκε (οργανωμένο σπορ) το παιδί σας; - v"],['Καθόλου', '1 – 2 φορές / βδ','3 – 5 φορές / βδ','6 – 7 φορές / βδ.'],'categorical2',[0,1,2,3])
dsactivity_habits = dsactivity_habits.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Είδος Άσκησης'
number = generate_descriptive_statistics(food_habits["Τι είδους άσκηση έκανε το παιδί σας την προηγούμενη βδομάδα; - v"],['Καμία', 'Χορός / μπαλέτο / ρυθμική','Μπάσκετ','Ποδόσφαιρο', 'Πολεμικές τέχνες','Κολύμβηση','Άλλη'],'categorical2',[0,1,2,3,4,5,7])
dsactivity_habits = dsactivity_habits.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Ύπνος'
number = generate_descriptive_statistics(food_habits["Πόσες ώρες την ημέρα περίπου κοιμάται το παιδί σας (μέσος όρος); - v"],['9 - 11  ώρες', '7 – 8 ώρες','< 7 ώρες'],'categorical2',[0,1,2])
dsactivity_habits = dsactivity_habits.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Έκθεση σε οθόνες - Καθημερινές'
number = generate_descriptive_statistics(food_habits["Πόσες ώρες περίπου την ημέρα (καθημερινές)  πέρασε το παιδί σας  μπροστά σε οθόνες; (π.χ. τηλεόραση, τάμπλετ, κινητό) - v"],['< 1 ώρα / μέρα', '1-2 ώρες / μέρα','2 – 3 ώρες / μέρα','3 – 4 ώρες /  μέρα', '> 4 ώρες / μέρα'],'categorical2',[0,1,2,3,4])
dsactivity_habits = dsactivity_habits.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Έκθεση σε οθόνες - ΣΚ'
number = generate_descriptive_statistics(food_habits["Πόσες ώρες περίπου την ημέρα (Σαββατοκύριακο ή αργίες)  πέρασε το παιδί σας μπροστά σε οθόνες; (π.χ. τηλεόραση, τάμπλετ, κινητό) - v"],['< 1 ώρα / μέρα', '1-2 ώρες / μέρα','2 – 3 ώρες / μέρα','3 – 4 ώρες /  μέρα', '> 4 ώρες / μέρα'],'categorical2',[0,1,2,3,4])
dsactivity_habits = dsactivity_habits.append({C:characteristic,N:number},ignore_index=True)



In [ ]:
dsactivity_habits

,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Σωματική Δραστηριότητα,"Καθόλου: 3(6.25%), < 30 λεπτά: 10(20.83%), 30 ..."
1,Οργανωμένη Άσκηση,"Καθόλου: 25(52.08%), 1 – 2 φορές / βδ: 9(18.75..."
2,Είδος Άσκησης,"Καμία: 18(37.5%), Χορός / μπαλέτο / ρυθμική: 1..."
3,Ύπνος,"9 - 11 ώρες: 23(47.92%), 7 – 8 ώρες: 23(47.92..."
4,Έκθεση σε οθόνες - Καθημερινές,"< 1 ώρα / μέρα: 4(8.33%), 1-2 ώρες / μέρα: 12(..."
5,Έκθεση σε οθόνες - ΣΚ,"< 1 ώρα / μέρα: 1(2.08%), 1-2 ώρες / μέρα: 4(8..."


In [2]:
# Read a particular sheet from two different Excel Files and place all the data in one DataFrame
post_foodpractices1 = retrieve_data('questionnaireData.xlsx',"post_foodpractices")
post_foodpractices2 = retrieve_data('questionnaireData_ctrlpre.xlsx',"post_foodpractices")
post_foodpractices = pd.concat([post_foodpractices1, post_foodpractices2], axis=0)
post_foodpractices.drop(['Unnamed: 0'],axis=1,inplace=True)
post_foodpractices.reset_index(drop=True, inplace=True)

# Process Scores (i.e., comma-separated values)
post_foodpractices_df = pd.DataFrame()

ColumnNames = ["Modeling","Coercive Control","Emotional Regulation/Food as reward","Healthy Eating Guidance",
               "Restriction","Pressure to eat","pid"] 
post_foodpractices = scores(post_foodpractices,ColumnNames)
characteristic = 'Γονεϊκές πρακτικές σίτισης'
number = generate_descriptive_statistics(post_foodpractices[["Modeling","Coercive Control","Emotional Regulation/Food as reward","Healthy Eating Guidance",
               "Restriction","Pressure to eat"]],
                ['Γονείς-πρότυπα','Καταναγκαστικός έλεγχος','Χρήση του φαγητού ως επιβράβευση','Ενθάρρυνση',
                 'Περιοριστική καθοδήγηση','Πίεση για φαγητό'],'continous',[])
post_foodpractices_df = post_foodpractices_df.append({C:characteristic,N:number},ignore_index=True)
post_foodpractices_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Γονεϊκές πρακτικές σίτισης,"Γονείς-πρότυπα: 4.26 ± 0.68, Καταναγκαστικός έ..."


In [3]:
# Read a particular sheet from two different Excel Files and place all the data in one DataFrame
psdq1 = retrieve_data('questionnaireData.xlsx','post_strengthsdifficulties')
psdq2 = retrieve_data('questionnaireData_ctrlpre.xlsx','post_strengthsdifficulties')
psdq = pd.concat([psdq1, psdq2], axis=0)
psdq.drop(['Unnamed: 0'],axis=1,inplace=True)
psdq.reset_index(drop=True, inplace=True)

# Process Scores (i.e., comma-separated values)
sdq_df = pd.DataFrame()

ColumnNames = ["Tot","Emo","Cond","Hyper","Peer","Beh","id"]
psdq = scores(psdq, ColumnNames)

def sdq_statistics(SDQ_name):
    Cat = ['Φυσιολογικό: ','Οριακό: ','Παθολογικό: ']
    l = 0
    for i in ['Nor','Bor','Abn']:
        if i=='Nor':
            temp = psdq[SDQ_name].str.find(i)
            number = Cat[l]  + str(len(temp[temp>-1])) + '(' + str(round(100*len(temp[temp>-1])/len(temp),2)) + '%)'
        else:
            temp = psdq[SDQ_name].str.find(i)
            number = number + ', ' + Cat[l] + str(len(temp[temp>-1])) + '(' + str(round(100*len(temp[temp>-1])/len(temp),2)) + '%)'
        l=l+1
    return number

characteristic = 'Δυνατότητες και δυσκολίες-Συνολικό'
number = sdq_statistics("Tot")
sdq_df = sdq_df.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Δυνατότητες και δυσκολίες-Συναισθηματικές διαταραχές'
number = sdq_statistics("Emo")
sdq_df = sdq_df.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Δυνατότητες και δυσκολίες-Προβλήματα συμπεριφοράς'
number = sdq_statistics("Cond")
sdq_df = sdq_df.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Δυνατότητες και δυσκολίες-Υπερκινητικότητα/διάσπαση προσοχής'
number = sdq_statistics("Hyper")
sdq_df = sdq_df.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Δυνατότητες και δυσκολίες-Προβλήματα ομηλίκων'
number = sdq_statistics("Peer")
sdq_df = sdq_df.append({C:characteristic,N:number},ignore_index=True)

characteristic = 'Δυνατότητες και δυσκολίες-Κοινωνική συμπεριφορά'
number = sdq_statistics("Beh")
sdq_df = sdq_df.append({C:characteristic,N:number},ignore_index=True)

sdq_df

,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Δυνατότητες και δυσκολίες-Συνολικό,"Φυσιολογικό: 29(80.56%), Οριακό: 2(5.56%), Παθ..."
1,Δυνατότητες και δυσκολίες-Συναισθηματικές διατ...,"Φυσιολογικό: 22(61.11%), Οριακό: 6(16.67%), Πα..."
2,Δυνατότητες και δυσκολίες-Προβλήματα συμπεριφοράς,"Φυσιολογικό: 21(58.33%), Οριακό: 6(16.67%), Πα..."
3,Δυνατότητες και δυσκολίες-Υπερκινητικότητα/διά...,"Φυσιολογικό: 30(83.33%), Οριακό: 2(5.56%), Παθ..."
4,Δυνατότητες και δυσκολίες-Προβλήματα ομηλίκων,"Φυσιολογικό: 27(75.0%), Οριακό: 4(11.11%), Παθ..."
5,Δυνατότητες και δυσκολίες-Κοινωνική συμπεριφορά,"Φυσιολογικό: 34(94.44%), Οριακό: 2(5.56%), Παθ..."


In [4]:
# Read a particular sheet from two different Excel Files and place all the data in one DataFrame
pmother1 = retrieve_data('questionnaireData.xlsx','post_mother')
pmother2 = retrieve_data('questionnaireData_ctrlpre.xlsx','post_mother')
pmother = pd.concat([pmother1, pmother2], axis=0)
pmother.drop(['Unnamed: 0'],axis=1,inplace=True)
pmother.reset_index(drop=True, inplace=True)

# Process Scores (i.e., comma-separated values)
pmother_df = pd.DataFrame()

ColumnNames = ["Ypostiriktiki","Afstiri","Epitreptiki","Aftarhiki","id"]
pmother = scores(pmother,ColumnNames)
characteristic = 'Γονεϊκός τύπος'
number = generate_descriptive_statistics(pmother[["Ypostiriktiki","Afstiri","Epitreptiki","Aftarhiki"]],['Υποστηρικτική','Αυστηρή','Επιτρεπτική','Αυταρχική'],'continous',[])
pmother_df = pmother_df.append({C:characteristic,N:number},ignore_index=True)

pmother_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Χαρακτηριστικό,Μέση τιμή ± Τυπική απόκλιση/Πλήθος
0,Γονεϊκός τύπος,"Υποστηρικτική: 4.38 ± 0.36, Αυστηρή: 3.43±0.57..."
